In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the District Platoon Staffing Exceptions data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Load the Calls for Service data
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Display the first few rows of each dataset
staffing_exceptions.head(), calls_for_service.head()

In [ ]:
# Convert date columns to datetime format
staffing_exceptions['Exception Date'] = pd.to_datetime(staffing_exceptions['Exception Date'])
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])

# Display the data types to confirm the conversion
staffing_exceptions.dtypes, calls_for_service.dtypes

In [ ]:
# Analyze the trends in staffing exceptions
staffing_trends = staffing_exceptions.groupby(['year', 'month']).size().reset_index(name='Staffing_Exceptions')

# Analyze the trends in calls for service
calls_trends = calls_for_service.groupby(['year', 'month']).size().reset_index(name='Calls_for_Service')

# Merge the two datasets on year and month
merged_data = pd.merge(staffing_trends, calls_trends, on=['year', 'month'])

# Display the merged data
merged_data.head()

In [ ]:
# Visualize the trends in staffing exceptions and calls for service
plt.figure(figsize=(15, 6))
sns.lineplot(data=merged_data, x='month', y='Staffing_Exceptions', hue='year', marker='o')
plt.title('Monthly Staffing Exceptions Trend')
plt.xlabel('Month')
plt.ylabel('Number of Staffing Exceptions')
plt.legend(title='Year')
plt.grid(True)
plt.show()

plt.figure(figsize=(15, 6))
sns.lineplot(data=merged_data, x='month', y='Calls_for_Service', hue='year', marker='o')
plt.title('Monthly Calls for Service Trend')
plt.xlabel('Month')
plt.ylabel('Number of Calls for Service')
plt.legend(title='Year')
plt.grid(True)
plt.show()

In [ ]:
# Prepare data for forecasting

# Extract day of year from 'Exception Date'
staffing_exceptions['DayOfYear'] = staffing_exceptions['Exception Date'].dt.dayofyear

# Group by day of year to get the total number of exceptions per day
exceptions_per_day = staffing_exceptions.groupby('DayOfYear').size().reset_index(name='Total_Exceptions')

# Extract day of year from 'Call Date'
calls_for_service['DayOfYear'] = calls_for_service['Call Date'].dt.dayofyear

# Group by day of year to get the total number of calls per day
calls_per_day = calls_for_service.groupby('DayOfYear').size().reset_index(name='Total_Calls')

# Merge the two datasets on day of year
merged_day_data = pd.merge(exceptions_per_day, calls_per_day, on='DayOfYear')

# Display the merged data
merged_day_data.head()

In [ ]:
# Visualize the merged data to identify patterns
plt.figure(figsize=(15, 6))
sns.lineplot(data=merged_day_data, x='DayOfYear', y='Total_Exceptions', label='Total Staffing Exceptions')
sns.lineplot(data=merged_day_data, x='DayOfYear', y='Total_Calls', label='Total Calls for Service')
plt.title('Staffing Exceptions and Calls for Service by Day of Year')
plt.xlabel('Day of Year')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Forecasting future scheduling challenges

# Seasonal Decompose for Staffing Exceptions
result_exceptions = seasonal_decompose(merged_day_data['Total_Exceptions'], model='additive', period=365)
result_exceptions.plot()
plt.show()

# Seasonal Decompose for Calls for Service
result_calls = seasonal_decompose(merged_day_data['Total_Calls'], model='additive', period=365)
result_calls.plot()
plt.show()

# Apply Exponential Smoothing model for forecasting
model_exceptions = ExponentialSmoothing(merged_day_data['Total_Exceptions'], seasonal='add', seasonal_periods=365).fit()
model_calls = ExponentialSmoothing(merged_day_data['Total_Calls'], seasonal='add', seasonal_periods=365).fit()

# Forecast for the next year
forecast_exceptions = model_exceptions.forecast(365)
forecast_calls = model_calls.forecast(365)

# Visualize the forecast
plt.figure(figsize=(15, 6))
plt.plot(forecast_exceptions, label='Forecasted Staffing Exceptions')
plt.plot(forecast_calls, label='Forecasted Calls for Service')
plt.title('Forecast of Staffing Exceptions and Calls for Service for the Next Year')
plt.xlabel('Day of Year')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Adjusting the forecasting approach due to insufficient data for seasonal decomposition

# Apply a simpler time series model for forecasting
from statsmodels.tsa.arima.model import ARIMA

# ARIMA model for Staffing Exceptions
model_exceptions_arima = ARIMA(merged_day_data['Total_Exceptions'], order=(5,1,0)).fit()
forecast_exceptions_arima = model_exceptions_arima.forecast(steps=365)

# ARIMA model for Calls for Service
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(5,1,0)).fit()
forecast_calls_arima = model_calls_arima.forecast(steps=365)

# Visualize the ARIMA forecast
plt.figure(figsize=(15, 6))
plt.plot(forecast_exceptions_arima, label='Forecasted Staffing Exceptions')
plt.plot(forecast_calls_arima, label='Forecasted Calls for Service')
plt.title('ARIMA Forecast of Staffing Exceptions and Calls for Service for the Next Year')
plt.xlabel('Day of Year')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 30-day forecast

# ARIMA model for Staffing Exceptions - 30 days forecast
forecast_exceptions_arima_30 = model_exceptions_arima.forecast(steps=30)

# ARIMA model for Calls for Service - 30 days forecast
forecast_calls_arima_30 = model_calls_arima.forecast(steps=30)

# Visualize the 30-day ARIMA forecast
plt.figure(figsize=(15, 6))
plt.plot(forecast_exceptions_arima_30, label='30-Day Forecasted Staffing Exceptions')
plt.plot(forecast_calls_arima_30, label='30-Day Forecasted Calls for Service')
plt.title('30-Day ARIMA Forecast of Staffing Exceptions and Calls for Service')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data['Total_Exceptions'][], label='Current Staffing Exceptions')
plt.plot(forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data['Total_Calls'][], label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast with current and forecasted data

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(last_10_days_exceptions.index, forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(last_10_days_calls.index, forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the error and visualizing the 10-day ARIMA forecast with current data

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'].tail(10)
last_10_days_calls = merged_day_data['Total_Calls'].tail(10)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(len(last_10_days_exceptions), len(last_10_days_exceptions) + 10), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(len(last_10_days_calls), len(last_10_days_calls) + 10), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Re-running the ARIMA model for 10-day forecast with current and forecasted data

# Re-define the merged_day_data dataframe
merged_day_data = pd.merge(daily_staffing_exceptions, daily_calls, left_index=True, right_index=True)

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'].tail(10)
last_10_days_calls = merged_day_data['Total_Calls'].tail(10)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(len(last_10_days_exceptions), len(last_10_days_exceptions) + 10), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(len(last_10_days_calls), len(last_10_days_calls) + 10), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Re-running the ARIMA model for 10-day forecast with current and forecasted data

# Re-define the merged_day_data dataframe
# Assuming daily_staffing_exceptions and daily_calls are already defined

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = daily_staffing_exceptions['Total_Exceptions'].tail(10)
last_10_days_calls = daily_calls['Total_Calls'].tail(10)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(len(last_10_days_exceptions), len(last_10_days_exceptions) + 10), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(len(last_10_days_calls), len(last_10_days_calls) + 10), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Re-running the ARIMA model for 10-day forecast with current and forecasted data

# Re-define the merged_day_data dataframe
# Assuming daily_staffing_exceptions and daily_calls are already defined

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = daily_staffing_exceptions['Total_Exceptions'].tail(10)
last_10_days_calls = daily_calls['Total_Calls'].tail(10)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(len(last_10_days_exceptions), len(last_10_days_exceptions) + 10), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(len(last_10_days_calls), len(last_10_days_calls) + 10), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Re-running the ARIMA model for 10-day forecast with current and forecasted data

# Re-define the merged_day_data dataframe
# Assuming daily_staffing_exceptions and daily_calls are already defined

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = daily_staffing_exceptions['Total_Exceptions'].tail(10)
last_10_days_calls = daily_calls['Total_Calls'].tail(10)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(len(last_10_days_exceptions), len(last_10_days_exceptions) + 10), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(len(last_10_days_calls), len(last_10_days_calls) + 10), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Reinitialize ARIMA models for Staffing Exceptions and Calls for Service

from statsmodels.tsa.arima.model import ARIMA

# Fit ARIMA model for Staffing Exceptions
model_exceptions_arima = ARIMA(merged_day_data['Total_Exceptions'], order=(5,1,0)).fit()

# Fit ARIMA model for Calls for Service
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(5,1,0)).fit()

# Forecast for the next 10 days
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Ensure the merged_day_data is available

# Check if merged_day_data is defined
try:
    print(merged_day_data.head())
except NameError:
    print("merged_day_data is not defined. Need to load or merge the data again.")

In [ ]:
# Load and merge the necessary data for ARIMA forecasting

# Load the Staffing Exceptions data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Load the Calls for Service data
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Merge the data on the appropriate key
merged_day_data = pd.merge(staffing_exceptions, calls_for_service, on='Date', how='inner')

# Display the first few rows of the merged data
merged_day_data.head()

In [ ]:
# Load and preprocess the Staffing Exceptions and Calls for Service data for ARIMA forecasting

import pandas as pd

# Load the Staffing Exceptions data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
# Preprocess the data
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions['Date'])
staffing_exceptions = staffing_exceptions.groupby('Date').sum()

# Load the Calls for Service data
calls_for_service = pd.read_csv('Calls_for_Service.csv')
# Preprocess the data
calls_for_service['Date'] = pd.to_datetime(calls_for_service['Date'])
calls_for_service = calls_for_service.groupby('Date').sum()

# Merge the data on the Date column
merged_day_data = pd.merge(staffing_exceptions, calls_for_service, on='Date', how='inner')

# Display the first few rows of the merged data
merged_day_data.head()

In [ ]:
# Load and preprocess the Staffing Exceptions and Calls for Service data for ARIMA forecasting

import pandas as pd

# Load the Staffing Exceptions data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
# Preprocess the data
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions.iloc[:, 0])
staffing_exceptions = staffing_exceptions.groupby('Date').sum()

# Load the Calls for Service data
calls_for_service = pd.read_csv('Calls_for_Service.csv')
# Preprocess the data
calls_for_service['Date'] = pd.to_datetime(calls_for_service.iloc[:, 0])
calls_for_service = calls_for_service.groupby('Date').sum()

# Merge the data on the Date column
merged_day_data = pd.merge(staffing_exceptions, calls_for_service, on='Date', how='inner')

# Display the first few rows of the merged data
merged_day_data.head()

In [ ]:
# Load and preprocess the Staffing Exceptions and Calls for Service data for ARIMA forecasting

import pandas as pd

# Load the Staffing Exceptions data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
# Preprocess the data
# Assuming the first column is the date column
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions.iloc[:, 0], errors='coerce')
staffing_exceptions.dropna(subset=['Date'], inplace=True)
staffing_exceptions = staffing_exceptions.groupby('Date').sum()

# Load the Calls for Service data
calls_for_service = pd.read_csv('Calls_for_Service.csv')
# Preprocess the data
# Assuming the first column is the date column
calls_for_service['Date'] = pd.to_datetime(calls_for_service.iloc[:, 0], errors='coerce')
calls_for_service.dropna(subset=['Date'], inplace=True)
calls_for_service = calls_for_service.groupby('Date').sum()

# Merge the data on the Date column
merged_day_data = pd.merge(staffing_exceptions, calls_for_service, on='Date', how='inner')

# Display the first few rows of the merged data
merged_day_data.head()

In [ ]:
# Re-attempt to load and preprocess the Staffing Exceptions and Calls for Service data

# Load the Staffing Exceptions data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
# Preprocess the data
# Assuming the first column is the date column
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions.iloc[:, 0], errors='coerce')
staffing_exceptions.dropna(subset=['Date'], inplace=True)
staffing_exceptions = staffing_exceptions.groupby('Date').sum()['# of Hours']

# Load the Calls for Service data
calls_for_service = pd.read_csv('Calls_for_Service.csv')
# Preprocess the data
# Assuming the first column is the date column
calls_for_service['Date'] = pd.to_datetime(calls_for_service.iloc[:, 0], errors='coerce')
calls_for_service.dropna(subset=['Date'], inplace=True)
calls_for_service = calls_for_service.groupby('Date').sum()['Total_Calls']

# Merge the data on the Date column
merged_day_data = pd.merge(staffing_exceptions, calls_for_service, on='Date', how='inner')

# Display the first few rows of the merged data
merged_day_data.head()

In [ ]:
# Re-attempt to load and preprocess the Staffing Exceptions and Calls for Service data

# Load the Staffing Exceptions data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
# Preprocess the data
# Assuming the first column is the date column
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions.iloc[:, 0], errors='coerce')
staffing_exceptions.dropna(subset=['Date'], inplace=True)
staffing_exceptions = staffing_exceptions.groupby('Date').sum()['# of Hours']

# Load the Calls for Service data
calls_for_service = pd.read_csv('Calls_for_Service.csv')
# Preprocess the data
# Assuming the first column is the date column
calls_for_service['Date'] = pd.to_datetime(calls_for_service.iloc[:, 0], errors='coerce')
calls_for_service.dropna(subset=['Date'], inplace=True)
# Assuming 'Total_Calls' is a column in the Calls for Service data
calls_for_service = calls_for_service.groupby('Date').sum()

# Merge the data on the Date column
merged_day_data = pd.merge(staffing_exceptions, calls_for_service, on='Date', how='inner')

# Display the first few rows of the merged data
merged_day_data.head()

In [ ]:
# Re-examine the data merging process for Staffing Exceptions and Calls for Service

# Check the columns of both dataframes
print("Staffing Exceptions Columns:", staffing_exceptions.columns)
print("Calls for Service Columns:", calls_for_service.columns)

# Check the first few rows of both dataframes
print("Staffing Exceptions Sample Data:")
print(staffing_exceptions.head())
print("\nCalls for Service Sample Data:")
print(calls_for_service.head())

In [ ]:
# Correcting the data examination process

# Reload the original data without grouping
staffing_exceptions_original = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
calls_for_service_original = pd.read_csv('Calls_for_Service.csv')

# Check the columns and first few rows of both dataframes
print("Staffing Exceptions Columns:", staffing_exceptions_original.columns)
print("Staffing Exceptions Sample Data:")
print(staffing_exceptions_original.head())
print("\nCalls for Service Columns:", calls_for_service_original.columns)
print("Calls for Service Sample Data:")
print(calls_for_service_original.head())

In [ ]:
# ARIMA model for 10-day forecast with current data trend

# Select the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(last_10_days_exceptions.index[-1]+1, last_10_days_exceptions.index[-1]+11), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(last_10_days_calls.index[-1]+1, last_10_days_calls.index[-1]+11), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the KeyError in the previous cell

# Re-fetching the last 10 days of current data for comparison
last_10_days_data = merged_day_data.tail(10)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_data.index, last_10_days_data['Total_Exceptions'], label='Current Staffing Exceptions')
plt.plot(range(last_10_days_data.index[-1]+1, last_10_days_data.index[-1]+11), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_data.index, last_10_days_data['Total_Calls'], label='Current Calls for Service')
plt.plot(range(last_10_days_data.index[-1]+1, last_10_days_data.index[-1]+11), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Re-initializing the ARIMA models due to NameError in the previous cell

# Importing necessary libraries
from statsmodels.tsa.arima.model import ARIMA

# Re-creating the ARIMA models
model_exceptions_arima = ARIMA(merged_day_data['Total_Exceptions'], order=(5,1,0)).fit()
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(5,1,0)).fit()

# Forecasting for the next 10 days
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualizing the 10-day forecast with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(range(merged_day_data.index[-1]+1, merged_day_data.index[-1]+11), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(range(merged_day_data.index[-1]+1, merged_day_data.index[-1]+11), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the KeyError and reinitializing the ARIMA models

# Importing necessary libraries
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import matplotlib.pyplot as plt

# Loading the merged data again
merged_day_data = pd.read_csv('/mnt/data/merged_day_data.csv')

# Re-creating the ARIMA models
model_exceptions_arima = ARIMA(merged_day_data['Total_Exceptions'], order=(5,1,0)).fit()
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(5,1,0)).fit()

# Forecasting for the next 10 days
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualizing the 10-day forecast with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(range(merged_day_data.index[-1]+1, merged_day_data.index[-1]+11), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(range(merged_day_data.index[-1]+1, merged_day_data.index[-1]+11), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Loading the required data files for ARIMA forecasting
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Loading the 'District_Platoon_Staffing_Exceptions.csv' and 'Calls_for_Service.csv'
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Preprocessing and merging the data for ARIMA forecasting
# ... (data preprocessing steps go here)

# Creating ARIMA models for forecasting
# ... (ARIMA model creation and fitting steps go here)

# Forecasting for the next 10 days
# ... (forecasting steps go here)

# Visualizing the 10-day forecast with current data trends
# ... (visualization steps go here)

In [ ]:
# Preprocessing the 'District_Platoon_Staffing_Exceptions.csv' data

# Converting date columns to datetime format
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions['Date'])

# Aggregating the total number of hours officers are off work each day
staffing_exceptions_daily = staffing_exceptions.groupby('Date').sum()

# Preprocessing the 'Calls_for_Service.csv' data

# Converting date columns to datetime format
calls_for_service['Date'] = pd.to_datetime(calls_for_service['Date'])

# Aggregating the total number of calls each day
calls_for_service_daily = calls_for_service.groupby('Date').sum()

# Merging the two datasets on the 'Date' column
merged_data = pd.merge(staffing_exceptions_daily, calls_for_service_daily, on='Date', how='inner')

# Displaying the first few rows of the merged data
merged_data.head()

In [ ]:
# Correcting the KeyError in the previous cell

# Checking the column names for both datasets
columns_staffing_exceptions = staffing_exceptions.columns
columns_calls_for_service = calls_for_service.columns

# Displaying the column names
columns_staffing_exceptions, columns_calls_for_service

In [ ]:
# Correcting the column names for preprocessing

# For 'District_Platoon_Staffing_Exceptions.csv'
staffing_exceptions['Exception Date'] = pd.to_datetime(staffing_exceptions['Exception Date'])
staffing_exceptions_daily = staffing_exceptions.groupby('Exception Date').sum()['# of Hours']

# For 'Calls_for_Service.csv'
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])
calls_for_service_daily = calls_for_service.groupby('Call Date').size()

# Merging the two datasets on the date column
merged_data = pd.merge(staffing_exceptions_daily, calls_for_service_daily, left_index=True, right_index=True, how='inner')
merged_data.columns = ['Total_Exceptions', 'Total_Calls']

# Displaying the first few rows of the merged data
merged_data.head()

In [ ]:
# ARIMA model for 10-day forecast with current data trends

# Select the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# Combine current data with forecasted data
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10])
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10])

# Visualize the combined current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions.index, combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls.index, combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the approach for combining current and forecasted data

# Resetting the index for current data to align with forecasted data
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:].reset_index(drop=True)
last_10_days_calls = merged_day_data['Total_Calls'][-10:].reset_index(drop=True)

# Combine current data with forecasted data
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10.reset_index(drop=True)], axis=0)
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10.reset_index(drop=True)], axis=0)

# Visualize the combined current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Revising the approach to correctly access the data for current trends and forecast

# Ensure the correct column names are used for current data
if 'Total_Exceptions' in merged_day_data.columns and 'Total_Calls' in merged_day_data.columns:
    # Select the last 10 days of current data for comparison
    last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:].reset_index(drop=True)
    last_10_days_calls = merged_day_data['Total_Calls'][-10:].reset_index(drop=True)

    # Combine current data with forecasted data
    combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10.reset_index(drop=True)], axis=0)
    combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10.reset_index(drop=True)], axis=0)

    # Visualize the combined current and forecasted data
    plt.figure(figsize=(15, 6))
    plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
    plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
    plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
    plt.xlabel('Day')
    plt.ylabel('Count')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print('Required data columns not found in the dataset.')

In [ ]:
# ARIMA model for 10-day forecast with current data trends

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(1, 11), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(1, 11), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the error and visualizing the 10-day ARIMA forecast with current data trends

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:].reset_index(drop=True)
last_10_days_calls = merged_day_data['Total_Calls'][-10:].reset_index(drop=True)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls, label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Revising the approach to correctly visualize the 10-day ARIMA forecast with current data trends

# Ensure the column names are correct
print(merged_day_data.columns)

# If the column names are different, adjust the column references accordingly

In [ ]:
# Adjusting the visualization approach based on the correct column names

# Extract the last 10 days of current data for '# of Hours'
last_10_days_hours = merged_day_data['# of Hours'][-10:].reset_index(drop=True)

# ARIMA model for '# of Hours' - 10 days forecast
forecast_hours_arima_10 = model_exceptions_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_hours, label='Current Staffing Exceptions (# of Hours)')
plt.plot(forecast_hours_arima_10, label='10-Day Forecasted Staffing Exceptions (# of Hours)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions (# of Hours) with Current Trends')
plt.xlabel('Day')
plt.ylabel('Hours')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Building and fitting ARIMA models for '# of Hours' and 'Calls for Service'

# Import ARIMA model
from statsmodels.tsa.arima.model import ARIMA

# Building ARIMA model for '# of Hours'
model_hours_arima = ARIMA(merged_day_data['# of Hours'], order=(5,1,0))
model_hours_arima_fit = model_hours_arima.fit()

# Building ARIMA model for 'Calls for Service'
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(5,1,0))
model_calls_arima_fit = model_calls_arima.fit()

# Forecasting the next 10 days for both '# of Hours' and 'Calls for Service'
forecast_hours_arima_10 = model_hours_arima_fit.forecast(steps=10)
forecast_calls_arima_10 = model_calls_arima_fit.forecast(steps=10)

# Visualizing the forecast
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_hours, label='Current Staffing Exceptions (# of Hours)')
plt.plot(forecast_hours_arima_10, label='10-Day Forecasted Staffing Exceptions (# of Hours)')
plt.plot(last_10_days_calls, label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Addressing the Schur decomposition solver error in ARIMA model fitting

# It's possible that the ARIMA model parameters need adjustment
# Trying a simpler model with different parameters

# Adjusting ARIMA model for '# of Hours'
model_hours_arima = ARIMA(merged_day_data['# of Hours'], order=(1,1,1))
model_hours_arima_fit = model_hours_arima.fit()

# Adjusting ARIMA model for 'Calls for Service'
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(1,1,1))
model_calls_arima_fit = model_calls_arima.fit()

# Forecasting the next 10 days for both '# of Hours' and 'Calls for Service'
forecast_hours_arima_10 = model_hours_arima_fit.forecast(steps=10)
forecast_calls_arima_10 = model_calls_arima_fit.forecast(steps=10)

# Visualizing the forecast
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_hours, label='Current Staffing Exceptions (# of Hours)')
plt.plot(forecast_hours_arima_10, label='10-Day Forecasted Staffing Exceptions (# of Hours)')
plt.plot(last_10_days_calls, label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Addressing the KeyError for 'Total_Calls' in ARIMA model fitting

# Checking the correct column name for calls for service
print(merged_day_data.columns)

# If the column name is different, adjust the ARIMA model fitting accordingly

In [ ]:
# Adjusting the ARIMA model fitting for the correct data

# It seems that the 'Total_Calls' column is not present in the merged_day_data
# We need to merge the calls for service data to the existing dataframe

# Merging calls for service data
merged_day_data = pd.merge(merged_day_data, calls_for_service_data, on=['year', 'month', 'day'], how='left')

# Re-checking the columns after merging
print(merged_day_data.columns)

In [ ]:
# Loading the 'Calls_for_Service.csv' data

# Importing necessary libraries
import pandas as pd

# Load the 'Calls_for_Service.csv' data
# Assuming the file is in the same directory as the notebook
# If not, provide the correct path
try:
    calls_for_service_data = pd.read_csv('Calls_for_Service.csv')
    print('Calls for Service data loaded successfully.')
except Exception as e:
    print('Error loading Calls for Service data:', e)

In [ ]:
# Merging the 'Calls_for_Service.csv' data with the existing dataframe

# Merging calls for service data
merged_day_data = pd.merge(merged_day_data, calls_for_service_data, on=['year', 'month', 'day'], how='left')

# Re-checking the columns after merging
print(merged_day_data.columns)

In [ ]:
# Building and fitting ARIMA models for '# of Hours' and 'Calls for Service'

# Import ARIMA model
from statsmodels.tsa.arima.model import ARIMA

# Building ARIMA model for '# of Hours'
model_hours_arima = ARIMA(merged_day_data['# of Hours'], order=(1,1,1))
model_hours_arima_fit = model_hours_arima.fit()

# Building ARIMA model for 'Calls for Service'
# Assuming 'Call Type' column represents the calls for service
model_calls_arima = ARIMA(merged_day_data['Call Type'], order=(1,1,1))
model_calls_arima_fit = model_calls_arima.fit()

# Forecasting the next 10 days for both '# of Hours' and 'Calls for Service'
forecast_hours_arima_10 = model_hours_arima_fit.forecast(steps=10)
forecast_calls_arima_10 = model_calls_arima_fit.forecast(steps=10)

# Visualizing the forecast
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_hours, label='Current Staffing Exceptions (# of Hours)')
plt.plot(forecast_hours_arima_10, label='10-Day Forecasted Staffing Exceptions (# of Hours)')
plt.plot(last_10_days_calls, label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast with current data trends

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(last_10_days_exceptions.index[-1] + 1, last_10_days_exceptions.index[-1] + 11), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(last_10_days_calls.index[-1] + 1, last_10_days_calls.index[-1] + 11), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the KeyError and visualizing the 10-day ARIMA forecast with current data trends

# Extract the last 10 days of current data for comparison
last_10_days_data = merged_day_data.tail(10)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_data.index, last_10_days_data['Total_Exceptions'], label='Current Staffing Exceptions')
plt.plot(range(last_10_days_data.index[-1] + 1, last_10_days_data.index[-1] + 11), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_data.index, last_10_days_data['Total_Calls'], label='Current Calls for Service')
plt.plot(range(last_10_days_data.index[-1] + 1, last_10_days_data.index[-1] + 11), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Re-initializing the ARIMA models due to NameError

# Re-fit the ARIMA models
model_exceptions_arima = ARIMA(merged_day_data['Total_Exceptions'], order=(5,1,0)).fit()
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(5,1,0)).fit()

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(range(merged_day_data.index[-1] + 1, merged_day_data.index[-1] + 11), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(range(merged_day_data.index[-1] + 1, merged_day_data.index[-1] + 11), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Checking the structure and column names of the merged dataset
merged_day_data.head()

In [ ]:
# Exploring the 'Call Type' column to understand its contents
merged_day_data['Call Type'].value_counts()

In [ ]:
# Exploring the '# of Hours' column to understand its role in the dataset
merged_day_data['# of Hours'].value_counts()

In [ ]:
# ARIMA model for 10-day forecast with current data trend

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Combine current data with forecasted data for visualization
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10])
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10])

# Visualize the combined data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions.index, combined_exceptions, label='Staffing Exceptions (Current and Forecasted)')
plt.plot(combined_calls.index, combined_calls, label='Calls for Service (Current and Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the KeyError and visualizing the data

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:].reset_index(drop=True)
last_10_days_calls = merged_day_data['Total_Calls'][-10:].reset_index(drop=True)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Combine current data with forecasted data for visualization
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10], ignore_index=True)
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10], ignore_index=True)

# Visualize the combined data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current and Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current and Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Revising the approach to address the KeyError

# Check the column names in the merged_day_data dataframe
display(merged_day_data.columns)

In [ ]:
# Adjusting the code to use the correct column names

# Extract the last 10 days of current data for comparison
# Assuming '# of Hours' represents staffing exceptions and 'Call Type' represents calls for service
last_10_days_exceptions = merged_day_data['# of Hours'][-10:].reset_index(drop=True)
last_10_days_calls = merged_day_data['Call Type'][-10:].reset_index(drop=True)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Combine current data with forecasted data for visualization
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10], ignore_index=True)
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10], ignore_index=True)

# Visualize the combined data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current and Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current and Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Revising the code to address the NameError

# Check if the ARIMA models for staffing exceptions and calls for service were previously defined
try:
    model_exceptions_arima
    model_calls_arima
except NameError:
    print("ARIMA models not defined. Need to build the models first.")

In [ ]:
# Building ARIMA models for Staffing Exceptions and Calls for Service

# Importing necessary libraries
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Building ARIMA model for Staffing Exceptions
model_exceptions_arima = ARIMA(merged_day_data['# of Hours'], order=(5,1,0))
model_exceptions_arima = model_exceptions_arima.fit()

# Building ARIMA model for Calls for Service
model_calls_arima = ARIMA(merged_day_data['Call Type'], order=(5,1,0))
model_calls_arima = model_calls_arima.fit()

# Display model summaries
print('Staffing Exceptions ARIMA Model Summary:')
display(model_exceptions_arima.summary())
print('\nCalls for Service ARIMA Model Summary:')
display(model_calls_arima.summary())

In [ ]:
# Revising the ARIMA model approach due to the LinAlgError

# Simplifying the ARIMA model orders
model_exceptions_arima = ARIMA(merged_day_data['# of Hours'], order=(1,0,1))
model_exceptions_arima = model_exceptions_arima.fit()

model_calls_arima = ARIMA(merged_day_data['Call Type'], order=(1,0,1))
model_calls_arima = model_calls_arima.fit()

# Display model summaries
print('Staffing Exceptions ARIMA Model Summary:')
display(model_exceptions_arima.summary())
print('\nCalls for Service ARIMA Model Summary:')
display(model_calls_arima.summary())

In [ ]:
# Revising the ARIMA model approach due to the ValueError

# Checking the data types and non-null counts of the columns
merged_day_data.info()

In [ ]:
# Investigating the data issue

# Checking the first few rows of the merged_day_data dataframe
display(merged_day_data.head())

In [ ]:
# Revisiting the data loading and merging process

# Loading the data from 'District_Platoon_Staffing_Exceptions.csv'
try:
    staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
    print('Staffing Exceptions Data Loaded Successfully')
except Exception as e:
    print('Error loading Staffing Exceptions Data:', e)

# Loading the data from 'Calls_for_Service.csv'
try:
    calls_for_service = pd.read_csv('Calls_for_Service.csv')
    print('Calls for Service Data Loaded Successfully')
except Exception as e:
    print('Error loading Calls for Service Data:', e)

# Merging the data
try:
    merged_day_data = pd.merge(staffing_exceptions, calls_for_service, on='Call Date', how='inner')
    print('Data Merged Successfully')
except Exception as e:
    print('Error merging data:', e)

# Displaying the first few rows of the merged data
display(merged_day_data.head())

In [ ]:
# Inspecting the 'Call Date' columns in both datasets

# Displaying the first few rows of the 'District_Platoon_Staffing_Exceptions.csv' data
display(staffing_exceptions.head())

# Displaying the first few rows of the 'Calls_for_Service.csv' data
display(calls_for_service.head())

In [ ]:
# ARIMA model for 10-day forecast with current data trends

# Combine current data with forecasted data for visualization
combined_exceptions = pd.concat([merged_day_data['Total_Exceptions'], forecast_exceptions_arima_10])
combined_calls = pd.concat([merged_day_data['Total_Calls'], forecast_calls_arima_10])

# Visualize the combined current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the approach for combining current data with forecasted data

# Get the last 10 days of current data
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# Combine current data with forecasted data
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10])
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10])

# Visualize the combined current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Revising the approach to correctly combine and visualize the data

# Ensure the column names are correct
print(merged_day_data.columns)

# If the column names are different, update them accordingly before combining

In [ ]:
# Correcting the column names for visualization

# It appears that the columns 'Total_Exceptions' and 'Total_Calls' are not present in the merged dataset
# We need to calculate these totals from the available data

# Calculate the total number of hours for exceptions and calls for service for the last 10 days
last_10_days_exceptions = merged_day_data['# of Hours'][-10:].cumsum()
last_10_days_calls = merged_day_data['Call Time (HH:MM:SS)'][-10:].cumsum()

# Combine current data with forecasted data
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10])
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10])

# Visualize the combined current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the approach for ARIMA forecasting

# First, we need to generate the ARIMA forecasts for the next 10 days

# Generate ARIMA forecast for Staffing Exceptions
model_exceptions_arima = ARIMA(merged_day_data['# of Hours'], order=(5,1,0))
model_exceptions_arima_fit = model_exceptions_arima.fit(disp=0)
forecast_exceptions_arima_10 = model_exceptions_arima_fit.forecast(steps=10)[0]

# Generate ARIMA forecast for Calls for Service
model_calls_arima = ARIMA(merged_day_data['Call Time (HH:MM:SS)'], order=(5,1,0))
model_calls_arima_fit = model_calls_arima.fit(disp=0)
forecast_calls_arima_10 = model_calls_arima_fit.forecast(steps=10)[0]

# Now, let's combine and visualize the current data with the forecasted data

In [ ]:
# Correcting the ARIMA model fitting

from statsmodels.tsa.arima.model import ARIMA

# Fit the ARIMA model for Staffing Exceptions without the 'disp' parameter
model_exceptions_arima = ARIMA(merged_day_data['# of Hours'], order=(5,1,0))
model_exceptions_arima_fit = model_exceptions_arima.fit()
forecast_exceptions_arima_10 = model_exceptions_arima_fit.forecast(steps=10)

# Fit the ARIMA model for Calls for Service without the 'disp' parameter
model_calls_arima = ARIMA(merged_day_data['Call Time (HH:MM:SS)'], order=(5,1,0))
model_calls_arima_fit = model_calls_arima.fit()
forecast_calls_arima_10 = model_calls_arima_fit.forecast(steps=10)

# Proceed to combine and visualize the current data with the forecasted data

In [ ]:
# Checking for stationarity in the time series data

from statsmodels.tsa.stattools import adfuller

# Perform Augmented Dickey-Fuller test on 'Total_Exceptions' and 'Total_Calls'
def check_stationarity(series, title):
    result = adfuller(series)
    print(f'ADF Statistic for {title}: {result[0]}')
    print(f'p-value: {result[1]}')
    if result[1] > 0.05:
        print(f'The {title} series is not stationary')
    else:
        print(f'The {title} series is stationary')

check_stationarity(merged_day_data['# of Hours'], 'Staffing Exceptions')
check_stationarity(merged_day_data['Call Time (HH:MM:SS)'], 'Calls for Service')

In [ ]:
# Simplifying the ARIMA model parameters

# Trying a simpler ARIMA model with lower order parameters
model_exceptions_arima_simplified = ARIMA(merged_day_data['# of Hours'], order=(1,0,1))
model_exceptions_arima_fit_simplified = model_exceptions_arima_simplified.fit()
forecast_exceptions_arima_10_simplified = model_exceptions_arima_fit_simplified.forecast(steps=10)

model_calls_arima_simplified = ARIMA(merged_day_data['Call Time (HH:MM:SS)'], order=(1,0,1))
model_calls_arima_fit_simplified = model_calls_arima_simplified.fit()
forecast_calls_arima_10_simplified = model_calls_arima_fit_simplified.forecast(steps=10)

# If successful, we will proceed to combine and visualize the current data with the forecasted data

In [ ]:
# Data Preprocessing Check

# Checking for missing values or anomalies in the data
print('Missing values in Staffing Exceptions:', merged_day_data['# of Hours'].isnull().sum())
print('Missing values in Calls for Service:', merged_day_data['Call Time (HH:MM:SS)'].isnull().sum())

# Checking for any anomalies in the data
print('Summary statistics for Staffing Exceptions:')
print(merged_day_data['# of Hours'].describe())
print('\nSummary statistics for Calls for Service:')
print(merged_day_data['Call Time (HH:MM:SS)'].describe())

In [ ]:
# ARIMA model for 10-day forecast

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the ARIMA forecasting approach

# Generate predictions for the next 10 days
forecast_exceptions_arima_10 = model_exceptions_arima.get_forecast(steps=10).predicted_mean
forecast_calls_arima_10 = model_calls_arima.get_forecast(steps=10).predicted_mean

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Adjusting the ARIMA forecasting method

# Generate predictions for the next 10 days using the correct ARIMA method
forecast_exceptions_arima_10 = model_exceptions_arima.predict(start=len(merged_day_data), end=len(merged_day_data)+9)
forecast_calls_arima_10 = model_calls_arima.predict(start=len(merged_day_data), end=len(merged_day_data)+9)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Revising the ARIMA model implementation for forecasting

# Fit the ARIMA model for Staffing Exceptions
model_exceptions_arima = ARIMA(merged_day_data['Total_Exceptions'], order=(5,1,0))
model_exceptions_arima_fit = model_exceptions_arima.fit()

# Fit the ARIMA model for Calls for Service
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(5,1,0))
model_calls_arima_fit = model_calls_arima.fit()

# Generate 10-day forecasts
forecast_exceptions_arima_10 = model_exceptions_arima_fit.forecast(steps=10)
forecast_calls_arima_10 = model_calls_arima_fit.forecast(steps=10)

# Visualize the 10-day forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Checking the structure of the merged_day_data dataset
display(merged_day_data.head())

In [ ]:
# Identifying relevant columns for ARIMA forecasting

columns_of_interest = ['Call Date', 'District', '# of Hours', 'Call Type']

# Displaying a sample of the data with the identified columns
merged_day_data[columns_of_interest].head()

In [ ]:
# ARIMA model for 10-day forecast with current and forecasted data

# Select the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# Combine current data with forecasted data
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10])
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10])

# Visualize the combined current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the KeyError and visualizing the combined current and forecasted data

# Select the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'].tail(10)
last_10_days_calls = merged_day_data['Total_Calls'].tail(10)

# Combine current data with forecasted data
combined_exceptions = pd.concat([last_10_days_exceptions.reset_index(drop=True), forecast_exceptions_arima_10.reset_index(drop=True)])
combined_calls = pd.concat([last_10_days_calls.reset_index(drop=True), forecast_calls_arima_10.reset_index(drop=True)])

# Visualize the combined current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Revising the code to address the KeyError issue

# Ensure the correct column names are used
print(merged_day_data.columns)

# If the column names are different, update the code accordingly

In [ ]:
# Updating the code with the correct column names for visualization

# Select the last 10 days of current data for '# of Hours'
last_10_days_hours = merged_day_data['# of Hours'].tail(10)

# Combine current data with forecasted data for Staffing Exceptions
combined_hours = pd.concat([last_10_days_hours.reset_index(drop=True), forecast_exceptions_arima_10.reset_index(drop=True)])

# Visualize the combined current and forecasted data for Staffing Exceptions
plt.figure(figsize=(15, 6))
plt.plot(combined_hours, label='Staffing Hours (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Hours with Current Trends')
plt.xlabel('Day')
plt.ylabel('Hours')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Generating the 10-day ARIMA forecast for Staffing Exceptions

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# Select the last 10 days of current data for '# of Hours'
last_10_days_hours = merged_day_data['# of Hours'].tail(10)

# Combine current data with forecasted data for Staffing Exceptions
combined_hours = pd.concat([last_10_days_hours.reset_index(drop=True), forecast_exceptions_arima_10.reset_index(drop=True)])

# Visualize the combined current and forecasted data for Staffing Exceptions
plt.figure(figsize=(15, 6))
plt.plot(combined_hours, label='Staffing Hours (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Hours with Current Trends')
plt.xlabel('Day')
plt.ylabel('Hours')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the AttributeError for the ARIMA forecast

# Ensure the ARIMA model is correctly defined and used
# If 'model_exceptions_arima' is not defined or incorrectly defined, it needs to be corrected

# Check if the model is defined
try:
    print(model_exceptions_arima)
except NameError:
    print("ARIMA model for Staffing Exceptions is not defined.")

In [ ]:
# Correcting the ARIMA model usage for forecasting

# Fit the ARIMA model for Staffing Exceptions
model_exceptions_arima_fitted = model_exceptions_arima.fit()

# Generate the 10-day forecast for Staffing Exceptions
forecast_exceptions_arima_10 = model_exceptions_arima_fitted.get_forecast(steps=10).predicted_mean

# Select the last 10 days of current data for '# of Hours'
last_10_days_hours = merged_day_data['# of Hours'].tail(10)

# Combine current data with forecasted data for Staffing Exceptions
combined_hours = pd.concat([last_10_days_hours.reset_index(drop=True), forecast_exceptions_arima_10.reset_index(drop=True)])

# Visualize the combined current and forecasted data for Staffing Exceptions
plt.figure(figsize=(15, 6))
plt.plot(combined_hours, label='Staffing Hours (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Hours with Current Trends')
plt.xlabel('Day')
plt.ylabel('Hours')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Addressing the LinAlgError in the ARIMA model

# Attempting a simpler ARIMA model with different parameters
from statsmodels.tsa.arima.model import ARIMA

# Define a simpler ARIMA model
model_exceptions_arima_simpler = ARIMA(merged_day_data['# of Hours'], order=(1, 1, 1))

# Fit the simpler ARIMA model
try:
    model_exceptions_arima_simpler_fitted = model_exceptions_arima_simpler.fit()
    print('Model fitted successfully')
except Exception as e:
    print('Error fitting model:', e)

In [ ]:
# Generating the 10-day forecast with the simpler ARIMA model

# Generate the 10-day forecast for Staffing Exceptions
forecast_exceptions_arima_10_simpler = model_exceptions_arima_simpler_fitted.get_forecast(steps=10).predicted_mean

# Select the last 10 days of current data for '# of Hours'
last_10_days_hours = merged_day_data['# of Hours'].tail(10)

# Combine current data with forecasted data for Staffing Exceptions
combined_hours_simpler = pd.concat([last_10_days_hours.reset_index(drop=True), forecast_exceptions_arima_10_simpler.reset_index(drop=True)])

# Visualize the combined current and forecasted data for Staffing Exceptions
plt.figure(figsize=(15, 6))
plt.plot(combined_hours_simpler, label='Staffing Hours (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Hours with Current Trends')
plt.xlabel('Day')
plt.ylabel('Hours')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast with current data trends

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(last_10_days_exceptions.index[-1] + 1, last_10_days_exceptions.index[-1] + 11), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(last_10_days_calls.index[-1] + 1, last_10_days_calls.index[-1] + 11), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the KeyError issue and visualizing the 10-day forecast with current trends

# Ensure the column names are correct
print(merged_day_data.columns)

# If the column names are different, update them accordingly in the following code

In [ ]:
# Updating the column names for the ARIMA forecast visualization

# Extract the last 10 days of current data for '# of Hours'
last_10_days_hours = merged_day_data['# of Hours'][-10:]

# ARIMA model for '# of Hours' - 10 days forecast
forecast_hours_arima_10 = model_exceptions_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_hours.index, last_10_days_hours, label='Current Staffing Exceptions')
plt.plot(range(last_10_days_hours.index[-1] + 1, last_10_days_hours.index[-1] + 11), forecast_hours_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast with current and forecasted data visualization

# Extract the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Combine current and forecasted data for visualization
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10])
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10])

# Visualize the combined data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions.index, combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls.index, combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast with current data trends

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Combine current data with forecasted data for visualization
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10])
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10])

# Visualize the combined current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the code to properly visualize the current and forecasted data

# Extract the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Total_Exceptions'].tail(10)
last_10_days_calls = merged_day_data['Total_Calls'].tail(10)

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Combine current and forecasted data for visualization
combined_exceptions = pd.concat([last_10_days_exceptions.reset_index(drop=True), forecast_exceptions_arima_10.reset_index(drop=True)])
combined_calls = pd.concat([last_10_days_calls.reset_index(drop=True), forecast_calls_arima_10.reset_index(drop=True)])

# Visualize the combined data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast with current data trends

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(last_10_days_exceptions.index, forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(last_10_days_calls.index, forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Checking the structure of the merged_day_data dataframe to identify the correct column names
merged_day_data.info()

In [ ]:
# Correcting the KeyError and re-running the ARIMA model for 10-day forecast with current data trends

# Ensure the correct column names are used
last_10_days_exceptions = merged_day_data['Staffing_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Calls_for_Service'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Combine current data with forecasted data for visualization
combined_exceptions = pd.concat([last_10_days_exceptions, forecast_exceptions_arima_10])
combined_calls = pd.concat([last_10_days_calls, forecast_calls_arima_10])

# Visualize the combined current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions.index, combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls.index, combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the KeyError issue

# Ensure the correct column names are used for current data trends
# Assuming 'Staffing_Exceptions' and 'Calls_for_Service' are the correct column names

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Staffing_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Calls_for_Service'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(last_10_days_exceptions.index, forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(last_10_days_calls.index, forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Re-checking the column names in the merged dataset
print(merged_day_data.columns)

In [ ]:
# Loading and checking the data from the 'District_Platoon_Staffing_Exceptions.csv' and 'Calls_for_Service.csv' files

# Load 'District_Platoon_Staffing_Exceptions.csv'
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
print('Staffing Exceptions Data:')
print(staffing_exceptions.head())

# Load 'Calls_for_Service.csv'
calls_for_service = pd.read_csv('Calls_for_Service.csv')
print('\nCalls for Service Data:')
print(calls_for_service.head())

In [ ]:
# Correcting the ARIMA model visualization with the right column names

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['# of Hours'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(last_10_days_exceptions.index, forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions with Current Trends')
plt.xlabel('Day')
plt.ylabel('Hours')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the KeyError issue

# Ensure the correct column names are used for current data trends
# Assuming 'Staffing_Exceptions' and 'Calls_for_Service' are the correct column names

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Staffing_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Calls_for_Service'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(last_10_days_exceptions.index, forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(last_10_days_calls.index, forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Merging the 'Staffing Exceptions' and 'Calls for Service' data on common date fields

# Convert 'Exception Date' and 'Call Date' to datetime format
staffing_exceptions['Exception Date'] = pd.to_datetime(staffing_exceptions['Exception Date'])
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])

# Aggregate data by date
staffing_exceptions_agg = staffing_exceptions.groupby('Exception Date').agg({'# of Hours': 'sum'}).rename(columns={'# of Hours': 'Total_Exceptions'})
calls_for_service_agg = calls_for_service.groupby('Call Date').size().rename('Total_Calls')

# Merge the aggregated data
merged_day_data = pd.merge(staffing_exceptions_agg, calls_for_service_agg, left_index=True, right_index=True, how='outer')

# Fill missing values with 0
merged_day_data.fillna(0, inplace=True)

# Display the first few rows of the merged data
merged_day_data.head()

In [ ]:
# Correcting the ARIMA model visualization with the right column names

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['# of Hours'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
# Correcting the forecast method usage
forecast_exceptions_arima_10 = model_exceptions_arima.predict(n_periods=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(last_10_days_exceptions.index, forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions with Current Trends')
plt.xlabel('Day')
plt.ylabel('Hours')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the ARIMA model forecast method

# Assuming the ARIMA models have been trained and are named as 'model_exceptions_arima' and 'model_calls_arima'

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.predict(n_periods=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.predict(n_periods=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(pd.date_range(start=last_10_days_exceptions.index[-1], periods=10, freq='D'), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(pd.date_range(start=last_10_days_calls.index[-1], periods=10, freq='D'), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Re-checking the column names in the merged dataset
print(merged_day_data.columns)

In [ ]:
# Re-checking the column names in the merged dataset
print(merged_day_data.columns)

In [ ]:
# ARIMA model for 10-day forecast with current and forecasted data visualization

from statsmodels.tsa.arima.model import ARIMA

# ARIMA model for Staffing Exceptions - 10 days forecast
model_exceptions_arima = ARIMA(merged_day_data['Total_Exceptions'], order=(5,1,0))
model_exceptions_arima_fit = model_exceptions_arima.fit()
forecast_exceptions_arima_10 = model_exceptions_arima_fit.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(5,1,0))
model_calls_arima_fit = model_calls_arima.fit()
forecast_calls_arima_10 = model_calls_arima_fit.forecast(steps=10)

# Extract the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Total_Exceptions'].tail(10)
last_10_days_calls = merged_day_data['Total_Calls'].tail(10)

# Combine current and forecasted data for visualization
combined_exceptions = pd.concat([last_10_days_exceptions.reset_index(drop=True), forecast_exceptions_arima_10.reset_index(drop=True)])
combined_calls = pd.concat([last_10_days_calls.reset_index(drop=True), forecast_calls_arima_10.reset_index(drop=True)])

# Visualize the combined data
plt.figure(figsize=(15, 6))
plt.plot(combined_exceptions, label='Staffing Exceptions (Current + Forecasted)')
plt.plot(combined_calls, label='Calls for Service (Current + Forecasted)')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the ARIMA model visualization with the right column names

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
# Correcting the forecast method usage
forecast_exceptions_arima_10 = model_exceptions_arima.predict(n_periods=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.predict(n_periods=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(pd.date_range(start=last_10_days_exceptions.index[-1], periods=10, freq='D'), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(pd.date_range(start=last_10_days_calls.index[-1], periods=10, freq='D'), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the ARIMA model visualization with the right column names

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
# Correcting the forecast method usage
forecast_exceptions_arima_10 = model_exceptions_arima.predict(n_periods=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.predict(n_periods=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(pd.date_range(start=last_10_days_exceptions.index[-1], periods=10, freq='D'), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(pd.date_range(start=last_10_days_calls.index[-1], periods=10, freq='D'), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the ARIMA model forecast method

# Assuming the ARIMA models have been trained and are named as 'model_exceptions_arima' and 'model_calls_arima'

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)[0]

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)[0]

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(pd.date_range(start=last_10_days_exceptions.index[-1], periods=10, freq='D'), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(pd.date_range(start=last_10_days_calls.index[-1], periods=10, freq='D'), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the ARIMA model forecast method using 'predict'

# Assuming the ARIMA models have been trained and are named as 'model_exceptions_arima' and 'model_calls_arima'

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.predict(n_periods=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.predict(n_periods=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(pd.date_range(start=last_10_days_exceptions.index[-1], periods=10, freq='D'), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(pd.date_range(start=last_10_days_calls.index[-1], periods=10, freq='D'), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast with current trends

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(last_10_days_exceptions.index, forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(last_10_days_calls.index, forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the ARIMA forecasting approach

# Generate forecast for the next 10 days
forecast_exceptions_arima_10 = model_exceptions_arima.get_forecast(steps=10)
forecast_calls_arima_10 = model_calls_arima.get_forecast(steps=10)

# Extract the forecasted values
forecasted_exceptions = forecast_exceptions_arima_10.predicted_mean
forecasted_calls = forecast_calls_arima_10.predicted_mean

# Visualize the 10-day forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], forecasted_exceptions, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(merged_day_data.index[-10:], forecasted_calls, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the ARIMA forecasting approach

# Re-fitting the ARIMA models
model_exceptions_arima = ARIMA(merged_day_data['Total_Exceptions'], order=(5,1,0)).fit()
model_calls_arima = ARIMA(merged_day_data['Total_Calls'], order=(5,1,0)).fit()

# Generate forecast for the next 10 days
forecast_exceptions_arima_10 = model_exceptions_arima.predict(start=len(merged_day_data), end=len(merged_day_data)+9)
forecast_calls_arima_10 = model_calls_arima.predict(start=len(merged_day_data), end=len(merged_day_data)+9)

# Visualize the 10-day forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(merged_day_data.index[-10:], forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast with current trends

# Extract the last 10 days of current data for comparison
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(range(10), forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(range(10), forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ARIMA model for 10-day forecast with current data trends

# Select the last 10 days of current data for visualization
last_10_days_exceptions = merged_day_data['Total_Exceptions'][-10:]
last_10_days_calls = merged_day_data['Total_Calls'][-10:]

# ARIMA model for Staffing Exceptions - 10 days forecast
forecast_exceptions_arima_10 = model_exceptions_arima.forecast(steps=10)

# ARIMA model for Calls for Service - 10 days forecast
forecast_calls_arima_10 = model_calls_arima.forecast(steps=10)

# Visualize the 10-day ARIMA forecast along with current data trends
plt.figure(figsize=(15, 6))
plt.plot(last_10_days_exceptions.index, last_10_days_exceptions, label='Current Staffing Exceptions')
plt.plot(last_10_days_exceptions.index, forecast_exceptions_arima_10, label='10-Day Forecasted Staffing Exceptions')
plt.plot(last_10_days_calls.index, last_10_days_calls, label='Current Calls for Service')
plt.plot(last_10_days_calls.index, forecast_calls_arima_10, label='10-Day Forecasted Calls for Service')
plt.title('10-Day ARIMA Forecast of Staffing Exceptions and Calls for Service with Current Trends')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Machine Learning Model for Forecasting

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Preparing the data for machine learning model
X = np.arange(len(merged_day_data)).reshape(-1, 1) # Day as feature
y_exceptions = merged_day_data['Total_Exceptions'].values
y_calls = merged_day_data['Total_Calls'].values

# Splitting the data into training and testing sets
X_train, X_test, y_train_exceptions, y_test_exceptions = train_test_split(X, y_exceptions, test_size=0.1, random_state=42)
X_train, X_test, y_train_calls, y_test_calls = train_test_split(X, y_calls, test_size=0.1, random_state=42)

# RandomForestRegressor model
model_exceptions = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train_exceptions)
model_calls = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train_calls)

# Forecasting for the next 10 days
future_days = np.arange(len(merged_day_data), len(merged_day_data) + 10).reshape(-1, 1)
forecast_exceptions_ml = model_exceptions.predict(future_days)
forecast_calls_ml = model_calls.predict(future_days)

# Visualize the trends and margins for current data and forecasted information
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data.index[-10:], y_exceptions[-10:], label='Current Staffing Exceptions')
plt.plot(future_days, forecast_exceptions_ml, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], y_calls[-10:], label='Current Calls for Service')
plt.plot(future_days, forecast_calls_ml, label='10-Day Forecasted Calls for Service')
plt.title('10-Day Forecast of Staffing Exceptions and Calls for Service with Current Trends (Machine Learning Model)')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Machine Learning Model for Forecasting

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Preparing the data
X = np.arange(len(merged_day_data)).reshape(-1, 1) # Day as feature
y_exceptions = merged_day_data['Total_Exceptions']
y_calls = merged_day_data['Total_Calls']

# Splitting the data into training and testing sets
X_train, X_test, y_train_exceptions, y_test_exceptions = train_test_split(X, y_exceptions, test_size=0.1, random_state=42)
X_train, X_test, y_train_calls, y_test_calls = train_test_split(X, y_calls, test_size=0.1, random_state=42)

# Training the model
model_exceptions = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train_exceptions)
model_calls = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train_calls)

# Forecasting for the next 10 days
X_future = np.arange(len(merged_day_data), len(merged_day_data)+10).reshape(-1, 1)
forecast_exceptions_ml = model_exceptions.predict(X_future)
forecast_calls_ml = model_calls.predict(X_future)

# Visualize the trends and forecast
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data.index[-10:], y_test_exceptions, label='Current Staffing Exceptions')
plt.plot(X_future, forecast_exceptions_ml, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], y_test_calls, label='Current Calls for Service')
plt.plot(X_future, forecast_calls_ml, label='10-Day Forecasted Calls for Service')
plt.title('10-Day Forecast of Staffing Exceptions and Calls for Service with Current Trends (Machine Learning Model)')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the code for Machine Learning Model Forecasting

# Forecasting for the next 10 days
X_future = np.arange(len(merged_day_data), len(merged_day_data)+10).reshape(-1, 1)
forecast_exceptions_ml = model_exceptions.predict(X_future)
forecast_calls_ml = model_calls.predict(X_future)

# Correcting the plot
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(merged_day_data.index[-10:].append(pd.date_range(start=merged_day_data.index[-1]+pd.Timedelta(days=1), periods=10)), forecast_exceptions_ml, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(merged_day_data.index[-10:].append(pd.date_range(start=merged_day_data.index[-1]+pd.Timedelta(days=1), periods=10)), forecast_calls_ml, label='10-Day Forecasted Calls for Service')
plt.title('10-Day Forecast of Staffing Exceptions and Calls for Service with Current Trends (Machine Learning Model)')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Adjusting the code for correct visualization

# Creating a date range for the forecast period
forecast_dates = pd.date_range(start=merged_day_data.index[-1] + pd.Timedelta(days=1), periods=10)

# Plotting the current and forecasted data
plt.figure(figsize=(15, 6))
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Exceptions'][-10:], label='Current Staffing Exceptions')
plt.plot(forecast_dates, forecast_exceptions_ml, label='10-Day Forecasted Staffing Exceptions')
plt.plot(merged_day_data.index[-10:], merged_day_data['Total_Calls'][-10:], label='Current Calls for Service')
plt.plot(forecast_dates, forecast_calls_ml, label='10-Day Forecasted Calls for Service')
plt.title('10-Day Forecast of Staffing Exceptions and Calls for Service with Current Trends (Machine Learning Model)')
plt.xlabel('Day')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Analysis of District_Platoon_Staffing_Exceptions.csv

# Load the data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Convert date columns to datetime format
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions['Date'])

# Group by date and sum the exceptions
staffing_exceptions_grouped = staffing_exceptions.groupby('Date').sum()

# Analysis of Calls_for_Service.csv

# Load the data
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Convert date columns to datetime format
calls_for_service['Date'] = pd.to_datetime(calls_for_service['Date'])

# Group by date and count the calls
calls_for_service_grouped = calls_for_service.groupby('Date').count()

# Correlation Analysis

# Merging the two datasets on Date
dataset_merged = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, on='Date')

# Calculating the correlation
staffing_calls_correlation = dataset_merged.corr()

# Display the correlation matrix
staffing_calls_correlation

In [ ]:
# Correcting the code for Analysis and Correlation

# Assuming 'Date' column exists in both datasets

# Load and preprocess District_Platoon_Staffing_Exceptions.csv
df_staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
df_staffing_exceptions['Date'] = pd.to_datetime(df_staffing_exceptions['Date'])
staffing_exceptions_grouped = df_staffing_exceptions.groupby('Date').sum()

# Load and preprocess Calls_for_Service.csv
df_calls_for_service = pd.read_csv('Calls_for_Service.csv')
df_calls_for_service['Date'] = pd.to_datetime(df_calls_for_service['Date'])
calls_for_service_grouped = df_calls_for_service.groupby('Date').count()

# Merge the datasets on 'Date'
merged_data = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, on='Date', how='inner')

# Correlation analysis
correlation_matrix = merged_data.corr()

# Display the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix for Staffing Exceptions and Calls for Service')
plt.show()

In [ ]:
# Checking the structure of the datasets

# Load the first few rows of District_Platoon_Staffing_Exceptions.csv
df_staffing_exceptions_sample = pd.read_csv('District_Platoon_Staffing_Exceptions.csv', nrows=5)

# Load the first few rows of Calls_for_Service.csv
df_calls_for_service_sample = pd.read_csv('Calls_for_Service.csv', nrows=5)

# Display the structure of both datasets
display(df_staffing_exceptions_sample)
display(df_calls_for_service_sample)

In [ ]:
# Correcting the code for Analysis and Correlation using the correct date columns

# Load and preprocess District_Platoon_Staffing_Exceptions.csv
df_staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
df_staffing_exceptions['Exception Date'] = pd.to_datetime(df_staffing_exceptions['Exception Date'])
staffing_exceptions_grouped = df_staffing_exceptions.groupby('Exception Date').sum()

# Load and preprocess Calls_for_Service.csv
df_calls_for_service = pd.read_csv('Calls_for_Service.csv')
df_calls_for_service['Call Date'] = pd.to_datetime(df_calls_for_service['Call Date'])
calls_for_service_grouped = df_calls_for_service.groupby('Call Date').count()

# Merge the datasets on the correct date columns
merged_data = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, left_on='Exception Date', right_on='Call Date', how='inner')

# Correlation analysis
correlation_matrix = merged_data.corr()

# Display the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix for Staffing Exceptions and Calls for Service')
plt.show()

In [ ]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Correcting the code for Analysis and Correlation using the correct date columns

# Load and preprocess District_Platoon_Staffing_Exceptions.csv
df_staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
df_staffing_exceptions['Exception Date'] = pd.to_datetime(df_staffing_exceptions['Exception Date'])
staffing_exceptions_grouped = df_staffing_exceptions.groupby('Exception Date').sum()

# Load and preprocess Calls_for_Service.csv
df_calls_for_service = pd.read_csv('Calls_for_Service.csv')
df_calls_for_service['Call Date'] = pd.to_datetime(df_calls_for_service['Call Date'])
calls_for_service_grouped = df_calls_for_service.groupby('Call Date').count()

# Merge the datasets on the correct date columns
merged_data = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, left_on='Exception Date', right_on='Call Date', how='inner')

# Correlation analysis
correlation_matrix = merged_data.corr()

# Display the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix for Staffing Exceptions and Calls for Service')
plt.show()

In [ ]:
# Analysis of District_Platoon_Staffing_Exceptions.csv

# Load the data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Convert date columns to datetime
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions['Date'])

# Group by date and sum the exceptions
staffing_exceptions_grouped = staffing_exceptions.groupby('Date').sum()

# Plotting the staffing exceptions over time
plt.figure(figsize=(15, 6))
plt.plot(staffing_exceptions_grouped.index, staffing_exceptions_grouped['Exceptions'], label='Staffing Exceptions')
plt.title('Staffing Exceptions Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Exceptions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the code for Analysis of District_Platoon_Staffing_Exceptions.csv

# Load the data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Check the column names to identify the correct date column
print(staffing_exceptions.columns)

In [ ]:
# Analysis of District_Platoon_Staffing_Exceptions.csv with Corrected Column Name

# Convert 'Exception Date' column to datetime
staffing_exceptions['Exception Date'] = pd.to_datetime(staffing_exceptions['Exception Date'])

# Group by 'Exception Date' and sum the '# of Hours'
staffing_exceptions_grouped = staffing_exceptions.groupby('Exception Date').sum()

# Plotting the staffing exceptions over time
plt.figure(figsize=(15, 6))
plt.plot(staffing_exceptions_grouped.index, staffing_exceptions_grouped['# of Hours'], label='Staffing Exceptions')
plt.title('Staffing Exceptions Over Time')
plt.xlabel('Date')
plt.ylabel('Total Hours of Exceptions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Analysis of Calls_for_Service.csv

# Load the data
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Convert date columns to datetime
calls_for_service['Date'] = pd.to_datetime(calls_for_service['Date'])

# Group by date and count the calls
service_calls_grouped = calls_for_service.groupby('Date').count()

# Plotting the calls for service over time
plt.figure(figsize=(15, 6))
plt.plot(service_calls_grouped.index, service_calls_grouped['Call ID'], label='Calls for Service')
plt.title('Calls for Service Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Calls')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the code for Analysis of Calls_for_Service.csv

# Load the data
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Check the column names to identify the correct date column
print(calls_for_service.columns)

In [ ]:
# Analysis of Calls_for_Service.csv with Corrected Column Name

# Convert 'Call Date' column to datetime
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])

# Group by 'Call Date' and count the calls
service_calls_grouped = calls_for_service.groupby('Call Date').count()

# Plotting the calls for service over time
plt.figure(figsize=(15, 6))
plt.plot(service_calls_grouped.index, service_calls_grouped['District'], label='Calls for Service')
plt.title('Calls for Service Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Calls')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correlation Analysis between Staffing Exceptions and Calls for Service

# Merging the two datasets on date
merged_data = pd.merge(staffing_exceptions_grouped, service_calls_grouped, left_index=True, right_index=True, how='inner')

# Calculating the correlation
correlation = merged_data[['# of Hours', 'District']].corr()

# Displaying the correlation matrix
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation between Staffing Exceptions and Calls for Service')
plt.show()

In [ ]:
# Statistical Modeling or Machine Learning for Future Crime Trends and Call Demands Prediction

# Preparing the data for modeling
# We will use the 'service_calls_grouped' data for predicting future call demands

# Extracting the target variable
y = service_calls_grouped['District']

# Creating a time series feature
X = np.arange(len(y)).reshape(-1, 1)

# Splitting the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using a simple linear regression model for demonstration
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

# Predicting on the test set
y_pred = model.predict(X_test)

# Evaluating the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

In [ ]:
# Time-Series Analysis for Forecasting Staffing Challenges

# We will use the 'staffing_exceptions_grouped' data for this analysis

# Importing necessary libraries for time-series analysis
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Defining the model
model = ARIMA(staffing_exceptions_grouped['# of Hours'], order=(5,1,0))

# Fitting the model
model_fit = model.fit()

# Forecasting the next 30 days
forecast = model_fit.forecast(steps=30)

# Plotting the forecast
plt.figure(figsize=(15, 6))
plt.plot(forecast, label='Forecasted Staffing Challenges')
plt.title('30-Day Forecast of Staffing Challenges')
plt.xlabel('Days')
plt.ylabel('Forecasted Hours of Staffing Exceptions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Time-Series Analysis for Forecasting Staffing Challenges

# We will use the 'staffing_exceptions_grouped' data for this analysis

# Importing necessary libraries for time-series analysis
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Defining the model
model = ARIMA(staffing_exceptions_grouped['# of Hours'], order=(5,1,0))

# Fitting the model
model_fit = model.fit()

# Forecasting the next 30 days
forecast = model_fit.forecast(steps=30)

# Plotting the forecast
plt.figure(figsize=(15, 6))
plt.plot(forecast, label='Forecasted Staffing Challenges')
plt.title('30-Day Forecast of Staffing Challenges')
plt.xlabel('Days')
plt.ylabel('Forecasted Hours of Staffing Exceptions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Developing a Model to Forecast Days with High Likelihood of Staffing Challenges and Calls for Service

# Importing necessary libraries
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Forecasting Staffing Challenges
staffing_model = ARIMA(staffing_exceptions_grouped['# of Hours'], order=(5,1,0))
staffing_model_fit = staffing_model.fit()
staffing_forecast = staffing_model_fit.forecast(steps=30)

# Forecasting Calls for Service
service_model = ARIMA(service_calls_grouped['District'], order=(5,1,0))
service_model_fit = service_model.fit()
service_forecast = service_model_fit.forecast(steps=30)

# Plotting the forecasts
plt.figure(figsize=(15, 6))
plt.plot(staffing_forecast, label='Forecasted Staffing Challenges')
plt.plot(service_forecast, label='Forecasted Calls for Service')
plt.title('30-Day Forecast of Staffing Challenges and Calls for Service')
plt.xlabel('Days')
plt.ylabel('Forecasted Values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Developing a Model to Forecast the Last 30 Days with High Likelihood of Staffing Challenges and Calls for Service

# Importing necessary libraries
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Forecasting Staffing Challenges for the Last 30 Days
staffing_model = ARIMA(staffing_exceptions_grouped['# of Hours'], order=(5,1,0))
staffing_model_fit = staffing_model.fit()
staffing_forecast = staffing_model_fit.forecast(steps=30)

# Forecasting Calls for Service for the Last 30 Days
service_model = ARIMA(service_calls_grouped['District'], order=(5,1,0))
service_model_fit = service_model.fit()
service_forecast = service_model_fit.forecast(steps=30)

# Plotting the forecasts
plt.figure(figsize=(15, 6))
plt.plot(staffing_forecast, label='Forecasted Staffing Challenges')
plt.plot(service_forecast, label='Forecasted Calls for Service')
plt.title('Last 30-Day Forecast of Staffing Challenges and Calls for Service')
plt.xlabel('Days')
plt.ylabel('Forecasted Values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Forecasting Staffing Challenges from 2020-11-30 to 2023-01-30

# Importing necessary libraries
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Selecting the relevant time period from the dataset
relevant_data = staffing_exceptions_grouped.loc['2020-11-30':'2023-01-30']

# Forecasting Staffing Challenges for the specified period
model = ARIMA(relevant_data['# of Hours'], order=(5,1,0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=30)

# Combining the dataset with the forecasting data
combined_data = pd.concat([relevant_data, forecast])

# Plotting the combined line chart
plt.figure(figsize=(15, 6))
plt.plot(combined_data, label='Staffing Challenges (Actual and Forecasted)')
plt.title('Staffing Challenges from 2020-11-30 to 2023-01-30')
plt.xlabel('Date')
plt.ylabel('Hours of Staffing Exceptions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Forecasting Staffing Challenges from 2020-11-30 to 2023-01-30 and Combining Actual and Forecasted Trends

# Importing necessary libraries
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Selecting the relevant time period from the dataset
relevant_data = staffing_exceptions_grouped.loc['2020-11-30':'2023-01-30']

# Forecasting Staffing Challenges for the specified period
model = ARIMA(relevant_data['# of Hours'], order=(5,1,0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=30)

# Combining the actual and forecasted data
combined_data = pd.concat([relevant_data, pd.Series(forecast, index=pd.date_range(start=relevant_data.index[-1], periods=30, freq='D'))])

# Plotting the combined chart
plt.figure(figsize=(15, 6))
plt.plot(relevant_data, label='Actual Staffing Challenges')
plt.plot(combined_data, label='Forecasted Staffing Challenges', linestyle='--')
plt.title('Staffing Challenges from 2020-11-30 to 2023-01-30 (Actual and Forecasted)')
plt.xlabel('Date')
plt.ylabel('Hours of Staffing Exceptions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Forecasting Staffing Challenges for 60 Days Between 2020-11-30 to 2023-01-30 and Combining Actual and Forecasted Trends

# Importing necessary libraries
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Selecting the relevant time period from the dataset
relevant_data = staffing_exceptions_grouped.loc['2020-11-30':'2023-01-30']

# Forecasting Staffing Challenges for the specified period
model = ARIMA(relevant_data['# of Hours'], order=(5,1,0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=60)

# Combining the actual and forecasted data
combined_data = pd.concat([relevant_data, pd.Series(forecast, index=pd.date_range(start=relevant_data.index[-1], periods=60, freq='D'))])

# Plotting the combined chart
plt.figure(figsize=(15, 6))
plt.plot(relevant_data, label='Actual Staffing Challenges')
plt.plot(combined_data, label='Forecasted Staffing Challenges', linestyle='--')
plt.title('Staffing Challenges for 60 Days Between 2020-11-30 to 2023-01-30 (Actual and Forecasted)')
plt.xlabel('Date')
plt.ylabel('Hours of Staffing Exceptions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Forecasting Staffing Challenges for 60 Days Between 2022-11-30 to 2023-01-30 and Combining Actual and Forecasted Trends

# Importing necessary libraries
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Selecting the relevant time period from the dataset
relevant_data = staffing_exceptions_grouped.loc['2022-11-30':'2023-01-30']

# Forecasting Staffing Challenges for the specified period
model = ARIMA(relevant_data['# of Hours'], order=(5,1,0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=60)

# Combining the actual and forecasted data
combined_data = pd.concat([relevant_data, pd.Series(forecast, index=pd.date_range(start=relevant_data.index[-1], periods=60, freq='D'))])

# Plotting the combined chart
plt.figure(figsize=(15, 6))
plt.plot(relevant_data, label='Actual Staffing Challenges')
plt.plot(combined_data, label='Forecasted Staffing Challenges', linestyle='--')
plt.title('Staffing Challenges for 60 Days Between 2022-11-30 to 2023-01-30 (Actual and Forecasted)')
plt.xlabel('Date')
plt.ylabel('Hours of Staffing Exceptions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Developing a Model to Forecast Days with High Likelihood of Staffing Challenges and Number of Calls for Service

# Importing necessary libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Loading and preparing the data
staffing_data = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
calls_data = pd.read_csv('Calls_for_Service.csv')

# Data preprocessing
staffing_data['Date'] = pd.to_datetime(staffing_data['Date'])
calls_data['Date'] = pd.to_datetime(calls_data['Date'])

# Aggregating data
staffing_grouped = staffing_data.groupby('Date').sum()
calls_grouped = calls_data.groupby('Date').sum()

# Merging datasets
combined_data = pd.merge(staffing_grouped, calls_grouped, on='Date')

# Developing ARIMA models for both datasets
model_staffing = ARIMA(combined_data['Staffing'], order=(5,1,0))
model_calls = ARIMA(combined_data['Calls'], order=(5,1,0))

# Fitting the models
model_staffing_fit = model_staffing.fit()
model_calls_fit = model_calls.fit()

# Forecasting for the next 30 days
forecast_staffing = model_staffing_fit.forecast(steps=30)
forecast_calls = model_calls_fit.forecast(steps=30)

# Combining actual and forecasted data
combined_forecast = pd.concat([combined_data, pd.DataFrame({'Forecasted Staffing': forecast_staffing, 'Forecasted Calls': forecast_calls}, index=pd.date_range(start=combined_data.index[-1], periods=30, freq='D'))])

# Plotting the trends
plt.figure(figsize=(15, 6))
plt.plot(combined_data['Staffing'], label='Actual Staffing')
plt.plot(combined_data['Calls'], label='Actual Calls')
plt.plot(combined_forecast['Forecasted Staffing'], label='Forecasted Staffing', linestyle='--')
plt.plot(combined_forecast['Forecasted Calls'], label='Forecasted Calls', linestyle='--')
plt.title('Forecasting Staffing Challenges and Calls for Service')
plt.xlabel('Date')
plt.ylabel('Number')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the Error and Developing a Model to Forecast Days with High Likelihood of Staffing Challenges and Number of Calls for Service

# Importing necessary libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Loading and preparing the data
# Assuming the date columns in both datasets are named 'Date'
try:
    staffing_data = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
    calls_data = pd.read_csv('Calls_for_Service.csv')
except Exception as e:
    print(f'Error loading data: {e}')
    raise

# Checking for the correct date column names
if 'Date' not in staffing_data.columns or 'Date' not in calls_data.columns:
    print('Date column not found in one or both datasets')
    raise KeyError('Date column not found')

# Data preprocessing
staffing_data['Date'] = pd.to_datetime(staffing_data['Date'])
calls_data['Date'] = pd.to_datetime(calls_data['Date'])

# Aggregating data
staffing_grouped = staffing_data.groupby('Date').sum()
calls_grouped = calls_data.groupby('Date').sum()

# Merging datasets
combined_data = pd.merge(staffing_grouped, calls_grouped, on='Date')

# Developing ARIMA models for both datasets
model_staffing = ARIMA(combined_data['Staffing'], order=(5,1,0))
model_calls = ARIMA(combined_data['Calls'], order=(5,1,0))

# Fitting the models
model_staffing_fit = model_staffing.fit()
model_calls_fit = model_calls.fit()

# Forecasting for the next 30 days
forecast_staffing = model_staffing_fit.forecast(steps=30)
forecast_calls = model_calls_fit.forecast(steps=30)

# Combining actual and forecasted data
combined_forecast = pd.concat([combined_data, pd.DataFrame({'Forecasted Staffing': forecast_staffing, 'Forecasted Calls': forecast_calls}, index=pd.date_range(start=combined_data.index[-1], periods=30, freq='D'))])

# Plotting the trends
plt.figure(figsize=(15, 6))
plt.plot(combined_data['Staffing'], label='Actual Staffing')
plt.plot(combined_data['Calls'], label='Actual Calls')
plt.plot(combined_forecast['Forecasted Staffing'], label='Forecasted Staffing', linestyle='--')
plt.plot(combined_forecast['Forecasted Calls'], label='Forecasted Calls', linestyle='--')
plt.title('Forecasting Staffing Challenges and Calls for Service')
plt.xlabel('Date')
plt.ylabel('Number')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Exploring the Data Columns to Identify the Correct Date Column

# Importing necessary libraries
import pandas as pd

# Loading the data
try:
    staffing_data = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
    calls_data = pd.read_csv('Calls_for_Service.csv')
except Exception as e:
    print(f'Error loading data: {e}')
    raise

# Displaying the column names for both datasets
columns_staffing = staffing_data.columns
columns_calls = calls_data.columns

print('Columns in Staffing Data:', columns_staffing)
print('Columns in Calls Data:', columns_calls)

In [ ]:
# Correcting the Date Column Names and Developing a Forecasting Model

# Data preprocessing
staffing_data['Exception Date'] = pd.to_datetime(staffing_data['Exception Date'])
calls_data['Call Date'] = pd.to_datetime(calls_data['Call Date'])

# Aggregating data
staffing_grouped = staffing_data.groupby('Exception Date').sum()
calls_grouped = calls_data.groupby('Call Date').sum()

# Merging datasets
combined_data = pd.merge(staffing_grouped, calls_grouped, left_index=True, right_index=True)

# Developing ARIMA models for both datasets
model_staffing = ARIMA(combined_data['# of Hours'], order=(5,1,0))
model_calls = ARIMA(combined_data['Call Type'], order=(5,1,0))

# Fitting the models
model_staffing_fit = model_staffing.fit()
model_calls_fit = model_calls.fit()

# Forecasting for the next 30 days
forecast_staffing = model_staffing_fit.forecast(steps=30)
forecast_calls = model_calls_fit.forecast(steps=30)

# Combining actual and forecasted data
combined_forecast = pd.concat([combined_data, pd.DataFrame({'Forecasted Staffing': forecast_staffing, 'Forecasted Calls': forecast_calls}, index=pd.date_range(start=combined_data.index[-1], periods=30, freq='D'))])

# Plotting the trends
plt.figure(figsize=(15, 6))
plt.plot(combined_data['# of Hours'], label='Actual Staffing')
plt.plot(combined_data['Call Type'], label='Actual Calls')
plt.plot(combined_forecast['Forecasted Staffing'], label='Forecasted Staffing', linestyle='--')
plt.plot(combined_forecast['Forecasted Calls'], label='Forecasted Calls', linestyle='--')
plt.title('Forecasting Staffing Challenges and Calls for Service')
plt.xlabel('Date')
plt.ylabel('Number')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the Date Column Names and Developing a Forecasting Model

# Data preprocessing
staffing_data['Exception Date'] = pd.to_datetime(staffing_data['Exception Date'])
calls_data['Call Date'] = pd.to_datetime(calls_data['Call Date'])

# Aggregating data
staffing_grouped = staffing_data.groupby('Exception Date').count()
calls_grouped = calls_data.groupby('Call Date').count()  # Using count instead of sum

# Merging datasets
combined_data = pd.merge(staffing_grouped, calls_grouped, left_index=True, right_index=True)

# Developing ARIMA models for both datasets
model_staffing = ARIMA(combined_data['# of Hours'], order=(5,1,0))
model_calls = ARIMA(combined_data['Call Type'], order=(5,1,0))

# Fitting the models
model_staffing_fit = model_staffing.fit()
model_calls_fit = model_calls.fit()

# Forecasting for the next 30 days
forecast_staffing = model_staffing_fit.forecast(steps=30)
forecast_calls = model_calls_fit.forecast(steps=30)

# Combining actual and forecasted data
combined_forecast = pd.concat([combined_data, pd.DataFrame({'Forecasted Staffing': forecast_staffing, 'Forecasted Calls': forecast_calls}, index=pd.date_range(start=combined_data.index[-1], periods=30, freq='D'))])

# Plotting the trends
plt.figure(figsize=(15, 6))
plt.plot(combined_data['# of Hours'], label='Actual Staffing')
plt.plot(combined_data['Call Type'], label='Actual Calls')
plt.plot(combined_forecast['Forecasted Staffing'], label='Forecasted Staffing', linestyle='--')
plt.plot(combined_forecast['Forecasted Calls'], label='Forecasted Calls', linestyle='--')
plt.title('Forecasting Staffing Challenges and Calls for Service')
plt.xlabel('Date')
plt.ylabel('Number')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Developing a Model to Forecast Days with High Likelihood of Staffing Challenges and Number of Calls for Service

# Importing necessary libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Merging staffing exceptions and calls for service data
merged_data = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, left_index=True, right_index=True)

# Forecasting using ARIMA model
model = ARIMA(merged_data, order=(5,1,0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=30)

# Combining the actual and forecasted data
combined_data = pd.concat([merged_data, pd.DataFrame(forecast, index=pd.date_range(start=merged_data.index[-1], periods=30, freq='D'))])

# Plotting the combined chart
plt.figure(figsize=(15, 6))
plt.plot(merged_data, label='Actual Data')
plt.plot(combined_data, label='Forecasted Data', linestyle='--')
plt.title('Forecast of Staffing Challenges and Calls for Service')
plt.xlabel('Date')
plt.ylabel('Number of Staffing Exceptions and Calls for Service')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the Model to Forecast Days with High Likelihood of Staffing Challenges and Number of Calls for Service

# Importing necessary libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Merging staffing exceptions and calls for service data
merged_data = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, left_index=True, right_index=True)

# Selecting a single column for forecasting (e.g., total number of calls for service)
total_calls = merged_data['Total Calls for Service']

# Forecasting using ARIMA model
model = ARIMA(total_calls, order=(5,1,0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=30)

# Combining the actual and forecasted data
combined_data = pd.concat([total_calls, pd.Series(forecast, index=pd.date_range(start=total_calls.index[-1], periods=30, freq='D'))])

# Plotting the combined chart
plt.figure(figsize=(15, 6))
plt.plot(total_calls, label='Actual Total Calls for Service')
plt.plot(combined_data, label='Forecasted Total Calls for Service', linestyle='--')
plt.title('Forecast of Total Calls for Service')
plt.xlabel('Date')
plt.ylabel('Number of Calls')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correcting the Model to Forecast Days with High Likelihood of Staffing Challenges and Number of Calls for Service

# Importing necessary libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Merging staffing exceptions and calls for service data
merged_data = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, left_index=True, right_index=True)

# Selecting a single column for forecasting (e.g., total number of staffing exceptions)
total_staffing_exceptions = merged_data['# of Hours']

# Forecasting using ARIMA model
model = ARIMA(total_staffing_exceptions, order=(5,1,0))
model_fit = model.fit()
forecast = model_fit.forecast(steps=30)

# Combining the actual and forecasted data
combined_data = pd.concat([total_staffing_exceptions, pd.Series(forecast, index=pd.date_range(start=total_staffing_exceptions.index[-1], periods=30, freq='D'))])

# Plotting the combined chart
plt.figure(figsize=(15, 6))
plt.plot(total_staffing_exceptions, label='Actual Staffing Exceptions')
plt.plot(combined_data, label='Forecasted Staffing Exceptions', linestyle='--')
plt.title('Forecast of Staffing Challenges')
plt.xlabel('Date')
plt.ylabel('Number of Staffing Exceptions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Correlation Analysis Between High Call Demand and Staffing Challenges

# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Merging staffing exceptions and calls for service data
merged_data = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, left_index=True, right_index=True)

# Calculating correlation
correlation = merged_data.corr()

# Plotting the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix between Staffing Challenges and Calls for Service')
plt.show()

In [ ]:
# Scheduling Optimization for Officer Staffing

# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets

# Function to simulate scheduling changes
def simulate_scheduling_change(vacation_days, sick_days, training_days):
    # Adjust the staffing data based on the input parameters
    # This is a simplified example and would need more detailed data for a real-world application
    adjusted_staffing = staffing_exceptions_grouped.copy()
    adjusted_staffing['Vacation'] -= vacation_days
    adjusted_staffing['Sick'] -= sick_days
    adjusted_staffing['Training'] -= training_days
    
    # Calculate the new total staffing exceptions
    adjusted_staffing['Total Exceptions'] = adjusted_staffing.sum(axis=1)
    
    # Plotting the adjusted staffing
    plt.figure(figsize=(15, 6))
    plt.plot(adjusted_staffing['Total Exceptions'], label='Adjusted Staffing')
    plt.title('Adjusted Staffing with Changes in Vacation, Sick, and Training Days')
    plt.xlabel('Date')
    plt.ylabel('Total Staffing Exceptions')
    plt.legend()
    plt.grid(True)
    plt.show()

# Interactive widget to adjust parameters
interact(simulate_scheduling_change, vacation_days=widgets.IntSlider(min=0, max=10, step=1, value=0), sick_days=widgets.IntSlider(min=0, max=10, step=1, value=0), training_days=widgets.IntSlider(min=0, max=10, step=1, value=0))

In [ ]:
# Advanced Scheduling Optimization for Officer Staffing

# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets

# Function to simulate scheduling adjustments based on current rule

def simulate_advanced_scheduling(current_staff, vacation_days, sick_days, training_days, high_demand_days):
    # Calculate the number of officers available each day
    available_officers = current_staff - (vacation_days + sick_days + training_days)
    # Adjust for high-demand days
    if high_demand_days:
        available_officers -= high_demand_days
    # Placeholder for more complex logic
    print(f'Available officers: {available_officers}')

# Creating interactive widgets
interact(simulate_advanced_scheduling, 
         current_staff=widgets.IntSlider(min=10, max=100, step=1, value=50), 
         vacation_days=widgets.IntSlider(min=0, max=30, step=1, value=10), 
         sick_days=widgets.IntSlider(min=0, max=30, step=1, value=5), 
         training_days=widgets.IntSlider(min=0, max=30, step=1, value=3),
         high_demand_days=widgets.IntSlider(min=0, max=30, step=1, value=5))

In [ ]:
# Analysis of Days with Most Officers Off and High Call Demands

# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Aggregating data to find days with most officers off
officers_off = staffing_exceptions.groupby('date').sum()

# Aggregating data to find days with high call demands
call_demands = calls_for_service.groupby('date').sum()

# Merging the two datasets
merged_data = pd.merge(officers_off, call_demands, on='date', how='inner')

# Identifying days with most officers off and high call demands
high_officers_off_high_calls = merged_data[(merged_data['officers_off'] > merged_data['officers_off'].quantile(0.75)) & (merged_data['calls'] > merged_data['calls'].quantile(0.75))]

# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls['officers_off'], label='Officers Off')
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls['calls'], label='Call Demands')
plt.legend()
plt.title('Days with Most Officers Off and High Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Correcting the Analysis for Days with Most Officers Off and High Call Demands

# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'staffing_exceptions' and 'calls_for_service' are the DataFrame names for the respective datasets
# If the names are different, please replace them with the correct DataFrame names

# Aggregating data to find days with most officers off
officers_off = staffing_exceptions.groupby('date').sum()

# Aggregating data to find days with high call demands
call_demands = calls_for_service.groupby('date').sum()

# Merging the two datasets
merged_data = pd.merge(officers_off, call_demands, on='date', how='inner')

# Identifying days with most officers off and high call demands
high_officers_off_high_calls = merged_data[(merged_data['officers_off'] > merged_data['officers_off'].quantile(0.75)) & (merged_data['calls'] > merged_data['calls'].quantile(0.75))]

# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls['officers_off'], label='Officers Off')
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls['calls'], label='Call Demands')
plt.legend()
plt.title('Days with Most Officers Off and High Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Loading the necessary datasets
import pandas as pd

# Load 'District_Platoon_Staffing_Exceptions.csv'
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Load 'Calls_for_Service.csv'
calls_for_service = pd.read_csv('Calls_for_Service.csv')

In [ ]:
# Preliminary Data Exploration

# Displaying the first few rows of 'staffing_exceptions'
print('Staffing Exceptions Data:')
print(staffing_exceptions.head())

# Displaying the first few rows of 'calls_for_service'
print('\nCalls for Service Data:')
print(calls_for_service.head())

In [ ]:
# Analysis of Days with Most Officers Off and High Call Demands

# Aggregating data to find days with most officers off
officers_off = staffing_exceptions.groupby('date').sum()

# Aggregating data to find days with high call demands
call_demands = calls_for_service.groupby('date').sum()

# Merging the two datasets
merged_data = pd.merge(officers_off, call_demands, on='date', how='inner')

# Identifying days with most officers off and high call demands
high_officers_off_high_calls = merged_data[(merged_data['officers_off'] > merged_data['officers_off'].quantile(0.75)) & (merged_data['calls'] > merged_data['calls'].quantile(0.75))]

# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls['officers_off'], label='Officers Off')
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls['calls'], label='Call Demands')
plt.legend()
plt.title('Days with Most Officers Off and High Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Correcting the Analysis for Days with Most Officers Off and High Call Demands

# Checking the column names for both datasets
columns_staffing = staffing_exceptions.columns
columns_calls = calls_for_service.columns

print('Staffing Exceptions Columns:', columns_staffing)
print('Calls for Service Columns:', columns_calls)

In [ ]:
# Revised Analysis for Days with Most Officers Off and High Call Demands

# Aggregating data to find days with most officers off
officers_off = staffing_exceptions.groupby('Exception Date').sum()['# of Hours']

# Aggregating data to find days with high call demands
call_demands = calls_for_service.groupby('Call Date').size()

# Merging the two datasets
merged_data = pd.merge(officers_off, call_demands, left_index=True, right_index=True, how='inner')

# Identifying days with most officers off and high call demands
high_officers_off_high_calls = merged_data[(merged_data['# of Hours'] > merged_data['# of Hours'].quantile(0.75)) & (merged_data[0] > merged_data[0].quantile(0.75))]

# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls['# of Hours'], label='Officers Off')
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls[0], label='Call Demands')
plt.legend()
plt.title('Days with Most Officers Off and High Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Correcting the Merge Error in the Analysis

# Renaming the series for call demands for merging
call_demands = call_demands.rename('Call Demands')

# Merging the two datasets again
merged_data = pd.merge(officers_off, call_demands, left_index=True, right_index=True, how='inner')

# Re-running the analysis
high_officers_off_high_calls = merged_data[(merged_data['# of Hours'] > merged_data['# of Hours'].quantile(0.75)) & (merged_data['Call Demands'] > merged_data['Call Demands'].quantile(0.75))]

# Plotting the results
plt.figure(figsize=(12, 6))
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls['# of Hours'], label='Officers Off')
plt.plot(high_officers_off_high_calls.index, high_officers_off_high_calls['Call Demands'], label='Call Demands')
plt.legend()
plt.title('Days with Most Officers Off and High Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Analysis of Days with Most Officers Off and High Call Demands

# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading the datasets
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Aggregating data to find days with most officers off
staffing_off = staffing_exceptions.groupby('Date')['Number_of_Officers_Off'].sum().reset_index()

# Aggregating data to find days with high call demands
call_demands = calls_for_service.groupby('Date')['Number_of_Calls'].sum().reset_index()

# Merging the two datasets
merged_data = pd.merge(staffing_off, call_demands, on='Date')

# Identifying days with most officers off and high call demands
high_risk_days = merged_data[(merged_data['Number_of_Officers_Off'] > merged_data['Number_of_Officers_Off'].quantile(0.75)) & (merged_data['Number_of_Calls'] > merged_data['Number_of_Calls'].quantile(0.75))]

# Plotting the results
plt.figure(figsize=(12, 6))
plt.scatter(high_risk_days['Date'], high_risk_days['Number_of_Officers_Off'], label='Officers Off')
plt.scatter(high_risk_days['Date'], high_risk_days['Number_of_Calls'], label='Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Days with Most Officers Off and High Call Demands')
plt.legend()
plt.show()

In [ ]:
# Correcting the Analysis for Days with Most Officers Off and High Call Demands

# Re-examining the datasets for the correct date columns
print(staffing_exceptions.columns)
print(calls_for_service.columns)

In [ ]:
# Revised Analysis of Days with Most Officers Off and High Call Demands

# Correcting the date column names
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions[['year', 'month', 'day']])
calls_for_service['Date'] = pd.to_datetime(calls_for_service[['year', 'month', 'day']])

# Aggregating data to find days with most officers off
staffing_off = staffing_exceptions.groupby('Date')['# of Hours'].sum().reset_index()

# Aggregating data to find days with high call demands
call_demands = calls_for_service.groupby('Date')['Call Type'].count().reset_index()

# Merging the two datasets
merged_data = pd.merge(staffing_off, call_demands, on='Date')

# Identifying days with most officers off and high call demands
high_risk_days = merged_data[(merged_data['# of Hours'] > merged_data['# of Hours'].quantile(0.80)) & (merged_data['Call Type'] > merged_data['Call Type'].quantile(0.80))]

# Plotting the results
plt.figure(figsize=(12, 6))
plt.scatter(high_risk_days['Date'], high_risk_days['# of Hours'], label='Officers Off')
plt.scatter(high_risk_days['Date'], high_risk_days['Call Type'], label='Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Days with Most Officers Off and High Call Demands')
plt.legend()
plt.show()
print(high_risk_days)

In [ ]:
# Comprehensive Analysis for 4 District

# Filtering data for 4 District
staffing_exceptions_4D = staffing_exceptions[staffing_exceptions['District'] == '4']
calls_for_service_4D = calls_for_service[calls_for_service['District'] == '4']

# Aggregating data to find days with most officers off in 4 District
staffing_off_4D = staffing_exceptions_4D.groupby('Date')['# of Hours'].sum().reset_index()

# Aggregating data to find days with high call demands in 4 District
call_demands_4D = calls_for_service_4D.groupby('Date')['Call Type'].count().reset_index()

# Merging the two datasets for 4 District
merged_data_4D = pd.merge(staffing_off_4D, call_demands_4D, on='Date')

# Identifying days with most officers off and high call demands in 4 District
high_risk_days_4D = merged_data_4D[(merged_data_4D['# of Hours'] > merged_data_4D['# of Hours'].quantile(0.75)) & (merged_data_4D['Call Type'] > merged_data_4D['Call Type'].quantile(0.75))]

# Plotting the results for 4 District
plt.figure(figsize=(12, 6))
plt.scatter(high_risk_days_4D['Date'], high_risk_days_4D['# of Hours'], label='Officers Off')
plt.scatter(high_risk_days_4D['Date'], high_risk_days_4D['Call Type'], label='Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Days with Most Officers Off and High Call Demands in 4 District')
plt.legend()
plt.show()

In [ ]:
# Correcting the Analysis for 4 District

# Re-loading the datasets
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Filtering data for 4 District
staffing_exceptions_4D = staffing_exceptions[staffing_exceptions['District'] == '4']
calls_for_service_4D = calls_for_service[calls_for_service['District'] == '4']

# Aggregating data to find days with most officers off in 4 District
staffing_off_4D = staffing_exceptions_4D.groupby('Date')['# of Hours'].sum().reset_index()

# Aggregating data to find days with high call demands in 4 District
call_demands_4D = calls_for_service_4D.groupby('Date')['Call Type'].count().reset_index()

# Merging the two datasets for 4 District
merged_data_4D = pd.merge(staffing_off_4D, call_demands_4D, on='Date')

# Identifying days with most officers off and high call demands in 4 District
high_risk_days_4D = merged_data_4D[(merged_data_4D['# of Hours'] > merged_data_4D['# of Hours'].quantile(0.75)) & (merged_data_4D['Call Type'] > merged_data_4D['Call Type'].quantile(0.75))]

# Plotting the results for 4 District
plt.figure(figsize=(12, 6))
plt.scatter(high_risk_days_4D['Date'], high_risk_days_4D['# of Hours'], label='Officers Off')
plt.scatter(high_risk_days_4D['Date'], high_risk_days_4D['Call Type'], label='Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Days with Most Officers Off and High Call Demands in 4 District')
plt.legend()
plt.show()

In [ ]:
# Correcting the Analysis for 4 District

# Re-loading the datasets
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Checking the column names for 'District' in both datasets
columns_staffing = staffing_exceptions.columns
columns_calls = calls_for_service.columns
print('Staffing Exceptions Columns:', columns_staffing)
print('Calls for Service Columns:', columns_calls)

In [ ]:
# Analysis for 4 District

# Filtering data for 4 District in Calls for Service
calls_for_service_4D = calls_for_service[calls_for_service['District'] == '4']

# Aggregating data to find days with high call demands in 4 District
call_demands_4D = calls_for_service_4D.groupby(['year', 'month', 'day'])['Call Type'].count().reset_index()

# Renaming columns for clarity
call_demands_4D.rename(columns={'Call Type': 'Call Count'}, inplace=True)

# Displaying the aggregated data
call_demands_4D.head()

In [ ]:
# Analysis for 4 District

# Filtering data for 4 District in Staffing Exceptions
staffing_exceptions_4D = staffing_exceptions[staffing_exceptions['Platoon'].str.contains('4')]

# Aggregating data to find days with most officers off in 4 District
staffing_off_4D = staffing_exceptions_4D.groupby(['year', 'month', 'day'])['# of Hours'].sum().reset_index()

# Renaming columns for clarity
staffing_off_4D.rename(columns={'# of Hours': 'Hours Off'}, inplace=True)

# Displaying the aggregated data
staffing_off_4D.head()

In [ ]:
# Merging Staffing and Call Demand Data for 4 District

# Merging the two datasets for 4 District
merged_data_4D = pd.merge(staffing_off_4D, call_demands_4D, on=['year', 'month', 'day'])

# Identifying days with most officers off and high call demands in 4 District
high_risk_days_4D = merged_data_4D[(merged_data_4D['Hours Off'] > merged_data_4D['Hours Off'].quantile(0.75)) & (merged_data_4D['Call Count'] > merged_data_4D['Call Count'].quantile(0.75))]

# Plotting the results for 4 District
plt.figure(figsize=(12, 6))
plt.scatter(high_risk_days_4D['year'], high_risk_days_4D['month'], high_risk_days_4D['day'], high_risk_days_4D['Hours Off'], label='Officers Off')
plt.scatter(high_risk_days_4D['year'], high_risk_days_4D['month'], high_risk_days_4D['day'], high_risk_days_4D['Call Count'], label='Call Demands')
plt.xlabel('Year')
plt.ylabel('Month, Day')
plt.title('Days with Most Officers Off and High Call Demands in 4 District')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Merging Staffing and Call Demand Data for 4 District

# Merging the two datasets for 4 District
merged_data_4D = pd.merge(staffing_off_4D, call_demands_4D, on=['year', 'month', 'day'])

# Identifying days with most officers off and high call demands in 4 District
high_risk_days_4D = merged_data_4D[(merged_data_4D['Hours Off'] > merged_data_4D['Hours Off'].quantile(0.75)) & (merged_data_4D['Call Count'] > merged_data_4D['Call Count'].quantile(0.75))]

# Plotting the results for 4 District
plt.figure(figsize=(12, 6))
plt.scatter(high_risk_days_4D['year'], high_risk_days_4D['month'], high_risk_days_4D['day'], high_risk_days_4D['Hours Off'], label='Officers Off')
plt.scatter(high_risk_days_4D['year'], high_risk_days_4D['month'], high_risk_days_4D['day'], high_risk_days_4D['Call Count'], label='Call Demands')
plt.xlabel('Year')
plt.ylabel('Month, Day')
plt.title('Days with Most Officers Off and High Call Demands in 4 District')
plt.legend()
plt.show()

In [ ]:
# Comprehensive Analysis for York Region and 4 District

# Loading the datasets
community_safety_data = pd.read_csv('Community_Safety_Data.csv')
calls_for_service = pd.read_csv('Calls_for_Service.csv')
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# 1) Crime Forecast for York Region
# a) Identifying the District most at risk of crime increase
# Analysis of crime data in York Region
# Integration of an external dataset for crime forecast model

# 2) Forecast of Future Scheduling Challenges for 4 District
# b) Identifying days with potential staffing challenges in 4 District
# Analysis of calls for service demand and staffing data in 4 District
# Consideration of historical crime data in 4 District

# Implementing the analysis and models

In [ ]:
# Crime Forecast for York Region

# Identifying the District most at risk of crime increase
# Analyzing crime data in York Region
crime_data_analysis = community_safety_data.groupby('District')['occ_type'].count().reset_index()
crime_data_analysis.sort_values(by='occ_type', ascending=False, inplace=True)

# Displaying the district with the highest crime occurrences
crime_data_analysis.head(1)

In [ ]:
# Correcting the Crime Forecast Analysis for York Region

# Checking the column names in the Community Safety Data
columns_community_safety = community_safety_data.columns
print('Community Safety Data Columns:', columns_community_safety)

In [ ]:
# Revised Crime Forecast Analysis for York Region

# Analyzing crime data in York Region using the correct column name
crime_data_analysis = community_safety_data.groupby('municipality')['occ_type'].count().reset_index()
crime_data_analysis.sort_values(by='occ_type', ascending=False, inplace=True)

# Displaying the municipality with the highest crime occurrences
crime_data_analysis.head(1)

In [ ]:
# Integrating External Dataset for Crime Forecast Model

# Loading the external dataset
external_data = pd.read_csv('Income_Population_Data.csv')

# Merging the crime data with the external dataset
crime_external_merged = pd.merge(crime_data_analysis, external_data, on='municipality')

# Analyzing the combined data for crime forecast
# Considering factors like income and population in the analysis
forecast_analysis = crime_external_merged.groupby('municipality').agg({'occ_type': 'sum', 'average_household_income': 'mean', 'population': 'mean'}).reset_index()

# Displaying the analysis results
forecast_analysis.head()

In [ ]:
# Checking Column Names in External Dataset

# Displaying the column names of the external dataset
columns_external_data = external_data.columns
print('Columns in External Dataset:', columns_external_data)

In [ ]:
# Revised Integration of External Dataset for Crime Forecast Model

# Correcting the column name for average household income
crime_external_merged = pd.merge(crime_data_analysis, external_data, on='municipality')

# Analyzing the combined data for crime forecast
forecast_analysis = crime_external_merged.groupby('municipality').agg({'occ_type': 'sum', 'avg_household_income': 'mean', 'population': 'mean'}).reset_index()

# Displaying the analysis results
forecast_analysis.head()

In [ ]:
# Identifying the Municipality Most at Risk of Crime Increase

# Analyzing the crime data with external factors
risk_analysis = forecast_analysis.copy()
risk_analysis['crime_rate'] = risk_analysis['occ_type'] / risk_analysis['population']
risk_analysis['income_per_capita'] = risk_analysis['avg_household_income'] / risk_analysis['population']

# Sorting the data to identify the municipality with the highest crime rate
risk_analysis.sort_values(by='crime_rate', ascending=False, inplace=True)

# Displaying the municipality with the highest crime rate
risk_analysis.head(1)

In [ ]:
# Revisiting the Crime Forecast Analysis for Identifying the Municipality Most at Risk

# Re-loading the combined crime and external dataset
crime_external_merged = pd.merge(community_safety_data, external_data, on='municipality')

# Calculating crime rate and income per capita
risk_analysis = crime_external_merged.groupby('municipality').agg({'occ_type': 'count', 'avg_household_income': 'mean', 'population': 'mean'}).reset_index()
risk_analysis['crime_rate'] = risk_analysis['occ_type'] / risk_analysis['population']
risk_analysis['income_per_capita'] = risk_analysis['avg_household_income'] / risk_analysis['population']

# Sorting the data to identify the municipality with the highest crime rate
risk_analysis.sort_values(by='crime_rate', ascending=False, inplace=True)

# Displaying the municipality with the highest crime rate
risk_analysis.head(1)

In [ ]:
# Revisiting the Crime Forecast Analysis with Correct Data

# Re-loading the crime data
community_safety_data = pd.read_csv('Community_Safety_Data.csv')

# Re-loading the external dataset
external_data = pd.read_csv('Income_Population_Data.csv')

# Merging the datasets
crime_external_merged = pd.merge(community_safety_data, external_data, on='municipality')

# Calculating crime rate and income per capita
risk_analysis = crime_external_merged.groupby('municipality').agg({'occ_type': 'count', 'avg_household_income': 'mean', 'population': 'mean'}).reset_index()
risk_analysis['crime_rate'] = risk_analysis['occ_type'] / risk_analysis['population']
risk_analysis['income_per_capita'] = risk_analysis['avg_household_income'] / risk_analysis['population']

# Sorting the data to identify the municipality with the highest crime rate
risk_analysis.sort_values(by='crime_rate', ascending=False, inplace=True)

# Displaying the municipality with the highest crime rate
risk_analysis.head(1)

In [ ]:
# Calculating Monthly Percentage Change in Crime Occurrences for Each Municipality

# Converting 'rep_date' to datetime format
community_safety_data['rep_date'] = pd.to_datetime(community_safety_data['rep_date'])

# Extracting year and month
community_safety_data['year'] = community_safety_data['rep_date'].dt.year
community_safety_data['month'] = community_safety_data['rep_date'].dt.month

# Grouping data by municipality, year, and month
monthly_crime_data = community_safety_data.groupby(['municipality', 'year', 'month']).agg({'occ_type': 'count'}).reset_index()

# Calculating monthly percentage change in crime occurrences
monthly_crime_data['monthly_pct_change'] = monthly_crime_data.groupby('municipality')['occ_type'].pct_change()

# Displaying the average monthly percentage change for each municipality
average_monthly_pct_change = monthly_crime_data.groupby('municipality')['monthly_pct_change'].mean().reset_index()

# Sorting the data to identify municipalities with the highest average monthly percentage change
average_monthly_pct_change.sort_values(by='monthly_pct_change', ascending=False, inplace=True)

average_monthly_pct_change

In [ ]:
# Re-loading the Community Safety Data
community_safety_data = pd.read_csv('Community_Safety_Data.csv')

# Calculating Monthly Percentage Change in Crime Occurrences for Each Municipality

# Converting 'rep_date' to datetime format
community_safety_data['rep_date'] = pd.to_datetime(community_safety_data['rep_date'])

# Extracting year and month
community_safety_data['year'] = community_safety_data['rep_date'].dt.year
community_safety_data['month'] = community_safety_data['rep_date'].dt.month

# Grouping data by municipality, year, and month
monthly_crime_data = community_safety_data.groupby(['municipality', 'year', 'month']).agg({'occ_type': 'count'}).reset_index()

# Calculating monthly percentage change in crime occurrences
monthly_crime_data['monthly_pct_change'] = monthly_crime_data.groupby('municipality')['occ_type'].pct_change()

# Displaying the average monthly percentage change for each municipality
average_monthly_pct_change = monthly_crime_data.groupby('municipality')['monthly_pct_change'].mean().reset_index()

# Sorting the data to identify municipalities with the highest average monthly percentage change
average_monthly_pct_change.sort_values(by='monthly_pct_change', ascending=False, inplace=True)

average_monthly_pct_change

In [ ]:
# Visualizing the Average Monthly Percentage Change in Crime Occurrences for Each Municipality

import matplotlib.pyplot as plt
import seaborn as sns

# Setting the plot style
sns.set(style='whitegrid')

# Creating the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='monthly_pct_change', y='municipality', data=average_monthly_pct_change, palette='coolwarm')

# Adding plot labels and title
plt.xlabel('Average Monthly Percentage Change')
plt.ylabel('Municipality')
plt.title('Average Monthly Percentage Change in Crime Occurrences by Municipality')

# Displaying the plot
plt.show()

In [ ]:
# Forecasting Future Scheduling Challenges for Each District

# Loading the District_Platoon_Staffing_Exceptions.csv
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Loading the Calls_for_Service.csv
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Converting dates to datetime format
staffing_exceptions['Date'] = pd.to_datetime(staffing_exceptions['Date'])
calls_for_service['Call_Date'] = pd.to_datetime(calls_for_service['Call_Date'])

# Grouping and aggregating data
staffing_exceptions_grouped = staffing_exceptions.groupby(['Date', 'District']).agg({'Hours_Off': 'sum'}).reset_index()
calls_for_service_grouped = calls_for_service.groupby(['Call_Date', 'District']).agg({'Call_Count': 'sum'}).reset_index()

# Merging the datasets
combined_data = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, left_on=['Date', 'District'], right_on=['Call_Date', 'District'], how='inner')

# Displaying the combined data
combined_data

In [ ]:
# Correcting the column names for the datasets

# Loading the District_Platoon_Staffing_Exceptions.csv
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Loading the Calls_for_Service.csv
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Checking the column names for both datasets
columns_staffing = staffing_exceptions.columns
columns_calls = calls_for_service.columns

# Displaying the column names
columns_staffing, columns_calls

In [ ]:
# Preparing and Merging the Data for Analysis

# Converting dates to datetime format in both datasets
staffing_exceptions['Exception Date'] = pd.to_datetime(staffing_exceptions['Exception Date'])
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])

# Grouping and aggregating data
staffing_exceptions_grouped = staffing_exceptions.groupby(['Exception Date', 'Platoon']).agg({'# of Hours': 'sum'}).reset_index()
calls_for_service_grouped = calls_for_service.groupby(['Call Date', 'District']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Merging the datasets
combined_data = pd.merge(staffing_exceptions_grouped, calls_for_service_grouped, left_on=['Exception Date', 'Platoon'], right_on=['Call Date', 'District'], how='inner')

# Displaying the combined data
combined_data

In [ ]:
# Analyzing the Data Separately Due to Merging Issue

# Analyzing Staffing Exceptions
staffing_exceptions_analysis = staffing_exceptions_grouped.groupby(['Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Analyzing Calls for Service
calls_for_service_analysis = calls_for_service_grouped.groupby(['Call Date']).agg({'Call_Count': 'sum'}).reset_index()

# Displaying the analyses
staffing_exceptions_analysis.head(), calls_for_service_analysis.head()

In [ ]:
# Visualizing Staffing Exceptions and Calls for Service Trends

import matplotlib.pyplot as plt
import seaborn as sns

# Setting plot style
sns.set(style='whitegrid')

# Plotting Staffing Exceptions
plt.figure(figsize=(15, 6))
plt.plot(staffing_exceptions_analysis['Exception Date'], staffing_exceptions_analysis['# of Hours'], label='Staffing Exceptions')
plt.plot(calls_for_service_analysis['Call Date'], calls_for_service_analysis['Call_Count'], label='Calls for Service')
plt.title('Staffing Exceptions and Calls for Service Trends')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# Loading the Community_Safety_Data.csv
crime_data = pd.read_csv('Community_Safety_Data.csv')

# Converting dates to datetime format in crime data
crime_data['rep_date'] = pd.to_datetime(crime_data['rep_date'])

# Grouping and aggregating crime data
crime_data_grouped = crime_data.groupby(['rep_date']).agg({'occ_type': 'count'}).rename(columns={'occ_type': 'Crime_Count'}).reset_index()

# Merging crime data with staffing exceptions and calls for service
combined_data_all = pd.merge(pd.merge(crime_data_grouped, staffing_exceptions_analysis, left_on='rep_date', right_on='Exception Date', how='inner'), calls_for_service_analysis, left_on='rep_date', right_on='Call Date', how='inner')

# Displaying the merged data
combined_data_all

In [ ]:
# Correcting the Data Merging Process

# Merging crime data with calls for service
crime_calls_merged = pd.merge(crime_data_grouped, calls_for_service_analysis, left_on='rep_date', right_on='Call Date', how='inner')

# Merging the above with staffing exceptions
final_combined_data = pd.merge(crime_calls_merged, staffing_exceptions_analysis, left_on='rep_date', right_on='Exception Date', how='inner')

# Displaying the final merged data
final_combined_data

In [ ]:
# Correcting the Data Merging Process

# Re-analyzing Calls for Service
calls_for_service_analysis = calls_for_service.groupby(['Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Re-analyzing Staffing Exceptions
staffing_exceptions_analysis = staffing_exceptions.groupby(['Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Merging crime data with calls for service
crime_calls_merged = pd.merge(crime_data_grouped, calls_for_service_analysis, left_on='rep_date', right_on='Call Date', how='inner')

# Merging the above with staffing exceptions
final_combined_data = pd.merge(crime_calls_merged, staffing_exceptions_analysis, left_on='rep_date', right_on='Exception Date', how='inner')

# Displaying the final merged data
final_combined_data

In [ ]:
# Loading Calls for Service and Staffing Exceptions Data

calls_for_service = pd.read_csv('Calls_for_Service.csv')
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Re-analyzing Calls for Service
calls_for_service_analysis = calls_for_service.groupby(['Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Re-analyzing Staffing Exceptions
staffing_exceptions_analysis = staffing_exceptions.groupby(['Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Merging crime data with calls for service
crime_calls_merged = pd.merge(crime_data_grouped, calls_for_service_analysis, left_on='rep_date', right_on='Call Date', how='inner')

# Merging the above with staffing exceptions
final_combined_data = pd.merge(crime_calls_merged, staffing_exceptions_analysis, left_on='rep_date', right_on='Exception Date', how='inner')

# Displaying the final merged data
final_combined_data

In [ ]:
# Correcting the Data Types for Merging

# Converting 'Call Date' and 'Exception Date' to datetime format
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])
staffing_exceptions['Exception Date'] = pd.to_datetime(staffing_exceptions['Exception Date'])

# Re-analyzing Calls for Service and Staffing Exceptions
calls_for_service_analysis = calls_for_service.groupby(['Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()
staffing_exceptions_analysis = staffing_exceptions.groupby(['Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Merging crime data with calls for service and staffing exceptions
final_combined_data = pd.merge(pd.merge(crime_data_grouped, calls_for_service_analysis, left_on='rep_date', right_on='Call Date', how='inner'), staffing_exceptions_analysis, left_on='rep_date', right_on='Exception Date', how='inner')

# Displaying the final merged data
final_combined_data

In [ ]:
# Correcting the Data Types for Merging

# Ensuring all dates are in the same format
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date']).dt.tz_localize(None)
staffing_exceptions['Exception Date'] = pd.to_datetime(staffing_exceptions['Exception Date']).dt.tz_localize(None)
crime_data_grouped['rep_date'] = crime_data_grouped['rep_date'].dt.tz_localize(None)

# Re-analyzing Calls for Service and Staffing Exceptions
calls_for_service_analysis = calls_for_service.groupby(['Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()
staffing_exceptions_analysis = staffing_exceptions.groupby(['Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Merging crime data with calls for service and staffing exceptions
final_combined_data = pd.merge(pd.merge(crime_data_grouped, calls_for_service_analysis, left_on='rep_date', right_on='Call Date', how='inner'), staffing_exceptions_analysis, left_on='rep_date', right_on='Exception Date', how='inner')

# Displaying the final merged data
final_combined_data

In [ ]:
# Importing Seaborn for Visualization
import seaborn as sns

# Calculating Correlations Among Crime Occurrence, Service Call, and Staffing Exceptions

# Calculating the correlation matrix
correlation_matrix = final_combined_data[['Crime_Count', 'Call_Count', '# of Hours']].corr()

# Displaying the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

In [ ]:
# Calculating Correlations Among Crime Occurrence, Service Call, and Staffing Exceptions

# Loading the Community_Safety_Data.csv
crime_data = pd.read_csv('Community_Safety_Data.csv')

# Converting dates to datetime format in crime data
crime_data['rep_date'] = pd.to_datetime(crime_data['rep_date'])

# Grouping and aggregating crime data
crime_data_grouped = crime_data.groupby(['rep_date']).agg({'occ_type': 'count'}).rename(columns={'occ_type': 'Crime_Count'}).reset_index()

# Merging the datasets for correlation analysis
combined_data_for_correlation = pd.merge(crime_data_grouped, staffing_exceptions_analysis, left_on='rep_date', right_on='Exception Date', how='inner')
combined_data_for_correlation = pd.merge(combined_data_for_correlation, calls_for_service_analysis, left_on='rep_date', right_on='Call Date', how='inner')

# Calculating the correlation matrix
correlation_matrix = combined_data_for_correlation[['Crime_Count', '# of Hours', 'Call_Count']].corr()

# Displaying the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix Among Crime Occurrence, Service Call, and Staffing Exceptions')
plt.show()

In [ ]:
# Correcting the Correlation Analysis

# Ensuring all datasets are properly loaded and grouped

# Loading and grouping the staffing exceptions data
staffing_exceptions = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')
staffing_exceptions['Exception Date'] = pd.to_datetime(staffing_exceptions['Exception Date'])
staffing_exceptions_grouped = staffing_exceptions.groupby(['Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Loading and grouping the calls for service data
calls_for_service = pd.read_csv('Calls_for_Service.csv')
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])
calls_for_service_grouped = calls_for_service.groupby(['Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Merging the datasets for correlation analysis
combined_data_for_correlation = pd.merge(crime_data_grouped, staffing_exceptions_grouped, left_on='rep_date', right_on='Exception Date', how='inner')
combined_data_for_correlation = pd.merge(combined_data_for_correlation, calls_for_service_grouped, left_on='rep_date', right_on='Call Date', how='inner')

# Calculating the correlation matrix
correlation_matrix = combined_data_for_correlation[['Crime_Count', '# of Hours', 'Call_Count']].corr()

# Displaying the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix Among Crime Occurrence, Service Call, and Staffing Exceptions')
plt.show()

In [ ]:
# Resolving the Merge Issue for Correlation Analysis

# Ensuring the date columns are in the same format for merging
staffing_exceptions_grouped['Exception Date'] = staffing_exceptions_grouped['Exception Date'].dt.tz_localize(None)
calls_for_service_grouped['Call Date'] = calls_for_service_grouped['Call Date'].dt.tz_localize(None)

# Merging the datasets for correlation analysis
combined_data_for_correlation = pd.merge(crime_data_grouped, staffing_exceptions_grouped, left_on='rep_date', right_on='Exception Date', how='inner')
combined_data_for_correlation = pd.merge(combined_data_for_correlation, calls_for_service_grouped, left_on='rep_date', right_on='Call Date', how='inner')

# Calculating the correlation matrix
correlation_matrix = combined_data_for_correlation[['Crime_Count', '# of Hours', 'Call_Count']].corr()

# Displaying the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix Among Crime Occurrence, Service Call, and Staffing Exceptions')
plt.show()

In [ ]:
# Correcting the Merge Issue for Correlation Analysis

# Ensuring the date columns are in the same format for merging
staffing_exceptions_grouped['Exception Date'] = staffing_exceptions_grouped['Exception Date'].dt.tz_localize(None)
calls_for_service_grouped['Call Date'] = calls_for_service_grouped['Call Date'].dt.tz_localize(None)

# Merging the datasets for correlation analysis
combined_data_for_correlation = pd.merge(crime_data_grouped, staffing_exceptions_grouped, left_on='rep_date', right_on='Exception Date', how='inner')
combined_data_for_correlation = pd.merge(combined_data_for_correlation, calls_for_service_grouped, left_on='rep_date', right_on='Call Date', how='inner')

# Calculating the correlation matrix
correlation_matrix = combined_data_for_correlation[['Crime_Count', '# of Hours', 'Call_Count']].corr()

# Displaying the correlation matrix
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix Among Crime Occurrence, Service Call, and Staffing Exceptions')
plt.show()

In [ ]:
# Exploratory Data Analysis (EDA) for Scheduling Challenges by District

# Loading the District_Platoon_Staffing_Exceptions.csv
district_staffing = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Converting dates to datetime format
district_staffing['Exception Date'] = pd.to_datetime(district_staffing['Exception Date'])

# Grouping data by date and district
staffing_by_date_district = district_staffing.groupby(['Exception Date', 'District']).agg({'# of Hours': 'sum'}).reset_index()

# Loading the Calls_for_Service.csv
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Converting dates to datetime format
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])

# Grouping data by date and district
calls_by_date_district = calls_for_service.groupby(['Call Date', 'District']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Merging the datasets for analysis
combined_data = pd.merge(staffing_by_date_district, calls_by_date_district, left_on=['Exception Date', 'District'], right_on=['Call Date', 'District'], how='inner')

# Visualizing the data
sns.lineplot(data=combined_data, x='Exception Date', y='# of Hours', hue='District', style='District', markers=True)
sns.lineplot(data=combined_data, x='Call Date', y='Call_Count', hue='District', style='District', markers=True)
plt.title('Staffing Challenges and Call Demands by District')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend(title='District')
plt.show()

In [ ]:
# Correcting the EDA for Scheduling Challenges by District

# Loading the District_Platoon_Staffing_Exceptions.csv
district_staffing = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Converting dates to datetime format
district_staffing['Exception Date'] = pd.to_datetime(district_staffing['Exception Date'])

# Grouping data by date
district_staffing_grouped = district_staffing.groupby(['Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Loading the Calls_for_Service.csv
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Converting dates to datetime format
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])

# Grouping data by date
calls_service_grouped = calls_for_service.groupby(['Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Merging the datasets for analysis
combined_data = pd.merge(district_staffing_grouped, calls_service_grouped, left_on='Exception Date', right_on='Call Date', how='inner')

# Visualizing the data
sns.lineplot(data=combined_data, x='Exception Date', y='# of Hours', label='Staffing Exceptions')
sns.lineplot(data=combined_data, x='Call Date', y='Call_Count', label='Call Demands')
plt.title('Staffing Challenges and Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# Importing necessary libraries for visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Correcting the EDA for Scheduling Challenges by District

# Loading the District_Platoon_Staffing_Exceptions.csv
district_staffing = pd.read_csv('District_Platoon_Staffing_Exceptions.csv')

# Converting dates to datetime format
district_staffing['Exception Date'] = pd.to_datetime(district_staffing['Exception Date'])

# Grouping data by date
district_staffing_grouped = district_staffing.groupby(['Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Loading the Calls_for_Service.csv
calls_for_service = pd.read_csv('Calls_for_Service.csv')

# Converting dates to datetime format
calls_for_service['Call Date'] = pd.to_datetime(calls_for_service['Call Date'])

# Grouping data by date
calls_service_grouped = calls_for_service.groupby(['Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Merging the datasets for analysis
combined_data = pd.merge(district_staffing_grouped, calls_service_grouped, left_on='Exception Date', right_on='Call Date', how='inner')

# Visualizing the data
sns.lineplot(data=combined_data, x='Exception Date', y='# of Hours', label='Staffing Exceptions')
sns.lineplot(data=combined_data, x='Call Date', y='Call_Count', label='Call Demands')
plt.title('Staffing Challenges and Call Demands')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# EDA for Scheduling Challenges Grouped by Platoon

# Grouping staffing data by Platoon and date
district_staffing_platoon_grouped = district_staffing.groupby(['Platoon', 'Exception Date']).count().reset_index()

# Visualizing the data by Platoon
plt.figure(figsize=(15, 6))
sns.lineplot(data=district_staffing_platoon_grouped, x='Exception Date', y='# of Hours', hue='Platoon')
plt.title('Staffing Challenges by Platoon')
plt.xlabel('Date')
plt.ylabel('Total Hours Off')
plt.legend(title='Platoon')
plt.show()

In [ ]:
# EDA for Scheduling Challenges Grouped by Platoon and Call Demands Grouped by District

# Grouping calls for service data by District and date
calls_service_district_grouped = calls_for_service.groupby(['District', 'Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Merging the datasets for analysis
combined_data_platoon_district = pd.merge(district_staffing_platoon_grouped, calls_service_district_grouped, left_on='Exception Date', right_on='Call Date', how='inner')

# Visualizing the data
plt.figure(figsize=(15, 6))
sns.lineplot(data=combined_data_platoon_district, x='Exception Date', y='# of Hours', hue='Platoon', style='District', markers=True)
plt.title('Staffing Challenges by Platoon and Call Demands by District')
plt.xlabel('Date')
plt.ylabel('Total Hours Off / Call Count')
plt.legend(title='Platoon / District')
plt.show()

In [ ]:
# EDA for Scheduling Challenges and Call Demands Grouped by District

# Grouping staffing data by District and date
district_staffing_grouped = district_staffing.groupby(['District', 'Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Merging the datasets for analysis
combined_data_district = pd.merge(district_staffing_grouped, calls_service_district_grouped, left_on=['District', 'Exception Date'], right_on=['District', 'Call Date'], how='inner')

# Visualizing the data
plt.figure(figsize=(15, 6))
sns.lineplot(data=combined_data_district, x='Exception Date', y='# of Hours', hue='District', style='District', markers=True)
sns.lineplot(data=combined_data_district, x='Call Date', y='Call_Count', hue='District', style='District', markers=True, dashes=True)
plt.title('Staffing Challenges and Call Demands Grouped by District')
plt.xlabel('Date')
plt.ylabel('Total Hours Off / Call Count')
plt.legend(title='District')
plt.show()

In [ ]:
# EDA for Scheduling Challenges and Call Demands Grouped by District

# Checking the columns in the datasets
district_staffing.columns, calls_for_service.columns

In [ ]:
# EDA for Scheduling Challenges and Call Demands Grouped by District

# Grouping staffing data by date
district_staffing_grouped = district_staffing.groupby(['Exception Date']).agg({'# of Hours': 'sum'}).reset_index()

# Grouping calls for service data by date
calls_service_grouped = calls_for_service.groupby(['Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Merging the datasets for analysis
combined_data_district = pd.merge(district_staffing_grouped, calls_service_grouped, left_on='Exception Date', right_on='Call Date', how='inner')

# Visualizing the data
plt.figure(figsize=(15, 6))
sns.lineplot(data=combined_data_district, x='Exception Date', y='# of Hours', label='Staffing Exceptions')
sns.lineplot(data=combined_data_district, x='Call Date', y='Call_Count', label='Call Demands', linestyle='--')
plt.title('Staffing Challenges and Call Demands Over Time')
plt.xlabel('Date')
plt.ylabel('Total Hours Off / Call Count')
plt.legend()
plt.show()

In [ ]:
# Time Series Analysis of Call Demands Grouped by District

# Grouping calls for service data by District and date
calls_service_district_grouped = calls_for_service.groupby(['District', 'Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Visualizing the data
plt.figure(figsize=(15, 6))
sns.lineplot(data=calls_service_district_grouped, x='Call Date', y='Call_Count', hue='District')
plt.title('Time Series of Call Demands Grouped by District')
plt.xlabel('Date')
plt.ylabel('Call Count')
plt.legend(title='District')
plt.show()

In [ ]:
# Time Series Analysis of Call Demands Grouped by District

# Grouping calls for service data by District and date
calls_service_district_grouped = calls_for_service.groupby(['District', 'Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Visualizing the data
g = sns.FacetGrid(calls_service_district_grouped, col='District', col_wrap=4, height=4, aspect=1.5)
g.map(sns.lineplot, 'Call Date', 'Call_Count')
g.set_titles('{col_name} District')
g.set_axis_labels('Date', 'Call Count')
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Time Series of Call Demands by District')
plt.show()

In [ ]:
# Time Series Analysis of Call Demands Grouped by District

# Grouping calls for service data by District and date
calls_service_district_grouped = calls_for_service.groupby(['District', 'Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Visualizing the data
g = sns.FacetGrid(calls_service_district_grouped, col='District', col_wrap=4, height=4, aspect=1.5)
g.map(sns.lineplot, 'Call Date', 'Call_Count')
g.set_titles('{col_name} District')
g.set_axis_labels('Date', 'Call Count')
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Time Series of Call Demands by District')
plt.show()

In [ ]:
# Time Series Analysis of Call Demands Grouped by District

# Grouping calls for service data by District and date
calls_service_district_grouped = calls_for_service.groupby(['District', 'Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Visualizing the data
g = sns.FacetGrid(calls_service_district_grouped, col='District', col_wrap=4, height=4, aspect=1.5)
g.map(sns.lineplot, 'Call Date', 'Call_Count')
g.set_titles('{col_name} District')
g.set_axis_labels('Date', 'Call Count')
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Time Series of Call Demands by District')
plt.show()

In [ ]:
# Time Series Analysis of Call Demands Grouped by District

# Grouping calls for service data by District and date
calls_service_district_grouped = calls_for_service.groupby(['District', 'Call Date']).agg({'Call Type': 'count'}).rename(columns={'Call Type': 'Call_Count'}).reset_index()

# Visualizing the data
g = sns.FacetGrid(calls_service_district_grouped, col='District', col_wrap=4, height=4, aspect=1.5)
g.map(sns.lineplot, 'Call Date', 'Call_Count')
g.set_titles('{col_name} District')
g.set_axis_labels('Date', 'Call Count')
plt.subplots_adjust(top=0.9)
g.fig.suptitle('Time Series of Call Demands by District')
plt.show()